<a href="https://colab.research.google.com/github/Justin-Hwang/EEG-AD-FTD-Detection/blob/main/Initial_Model_Training_Sanity_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# MyDrive 루트에 어떤 폴더들이 있는지 확인
!ls /content/drive/MyDrive

'0405 Research report.gdoc'
'2022 한화 글로벌 인턴십 최종 발표 자료.pptx'
'2023 Spring Semester'
'2024 Fall Lab Research'
'2024 Fall Semester'
'2024 Hanwha Global Internship'
'2024 Resume'
'2024 Spring Semester'
'2024 생각정리.gdoc'
'2024 신년 계획.gsheet'
 2025_Lab_Research
'2025 신년 계획.gsheet'
'Act Utilitarianism.gdoc'
 A_LAB3_wireless_communication.gdoc
'A_Lab7&8_Carrer_Selep_Hwang.gdoc'
'Anheuser Busch final round interview.gdoc'
'Anheuser-Busch ML Engineer Qualification.gdoc'
 Assignment2.zip
'Assignment 4.gdoc'
'Assignment 7 [Output Screenshots].gdoc'
 Assignments
'Assignments 2.gdoc'
'Berkeley Personal History.gdoc'
'Big Data Final Project'
'Big Data.zip'
'B_Lab 2_Justin Hwang .gdoc'
'Bondit Internship'
'Bonus Point.gdoc'
'Career development'
'Coding Test'
'Colab Notebooks'
'Columbia University Office Hour.gdoc'
'Columbia University Video Interview.gdoc'
'Columbia Univ Interview Preparation Answer sheet.gdoc'
'Copy of cover-letter-guide-and-samples.gdoc'
'Cop

In [ ]:
# 그중에 2025 Lab Research 폴더 안을 확인
!ls "/content/drive/MyDrive/2025_Lab_Research"

'Colab Files'			  eeg_holdout.db
'Data Preparation.gdoc'		  eeg_holdout_fixed_1.db
 eeg_dataset.py			  eeg_optuna_trial_1.db
 EEGformer_model_training.ipynb   eeg_optuna_trial_2.db
 eegformer_optuna_cv_3.db	  eeg_optuna_trial_3.db
 eegformer_optuna_cv_4.db	 'EEG Transformer Architecture.gdoc'
 eegformer_optuna_cv_5.db	 'Lab Info'
 eeg_holdout-1.db		 'Lab Research Paper Review'
 eeg_holdout-2.db		 'Meeting Note.gdoc'
 eeg_holdout-3.db		  model-data
 eeg_holdout-4.db		  model-data.zip
 eeg_holdout-5.db		  models_depracated.py
 eeg_holdout-6.db		  models.py
 eeg_holdout-7.db		  Practice_Note0.ipynb
 eeg_holdout-8.db		  __pycache__
 eeg_holdout-9.db		  Untitled


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/2025_Lab_Research')

In [ ]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on", DEVICE)  # → “cuda” 가 뜨면 GPU 정상

Running on cuda


In [ ]:
import wandb
wandb.login()  # 첫 실행 시 API 키 입력

wandb: Currently logged in as: jh8032 (jh8032-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!pip install optuna
!pip install wandb
!pip install mne

### 1. Model Overfit Test

In [ ]:
import os
import json

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from eeg_dataset import EEGDataset
from models import EEGformer

def main():
    # ─── 설정 ──────────────────────────────────────────
    DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
    LABEL_FILE  = "labels.json"
    BATCH_SIZE  = 32
    LR          = 1e-3
    NUM_ITERS   = 100
    KERNEL_SIZE = 10      # ODCM Kernel Size
    NUM_FILTERS = 120     # ODCM Filter (C)
    NUM_HEADS   = 4       # Transformer Heads
    NUM_BLOCKS  = 2       # Transformer Blocks
    NUM_SEGMENTS= 15      # TTM time segments (M)
    NUM_CLASSES = 3       # Class

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # ─── Meta Data Load─────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]

    # ─── Dataset & DataLoader ───────────────────────────
    dataset    = EEGDataset(DATA_DIR, train_meta)
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    # ─── Extract One Batch ─────────────────────────────────
    X_small, y_small = next(iter(dataloader))
    X_small, y_small = X_small.to(device), y_small.to(device)
    B, S, L = X_small.shape  # Batch, Channels, Time-length

    print(f"Overfit Test Batch Shape: X_small={X_small.shape}, y_small={y_small.shape} on {device}")

    # ─── Model, Loss, Optimizer ────────────────────────
    model = EEGformer(
        in_channels  = S,
        input_length = L,
        kernel_size  = KERNEL_SIZE,
        num_filters  = NUM_FILTERS,
        num_heads    = NUM_HEADS,
        num_blocks   = NUM_BLOCKS,
        num_segments = NUM_SEGMENTS,
        num_classes  = NUM_CLASSES,
    ).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    # ─── Overfit Training Loop ─────────────────────────────────
    model.train()
    for i in range(1, NUM_ITERS+1):
        optimizer.zero_grad()
        logits = model(X_small)            # [BATCH_SIZE, NUM_CLASSES]
        loss   = criterion(logits, y_small)
        loss.backward()
        optimizer.step()

        if i == 1 or i % 10 == 0:
            # 배치 정확도 계산
            with torch.no_grad():
                preds = logits.argmax(dim=1)
                acc   = (preds == y_small).float().mean().item() * 100
            print(f"Iter {i:03d} | loss = {loss.item():.6f} | acc = {acc:5.2f}%")

    print("Finished overfit test.")

if __name__ == "__main__":
    main()

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory
Overfit Test Batch Shape: X_small=torch.Size([32, 19, 1425]), y_small=torch.Size([32]) on cuda
Iter 001 | loss = 1.102822 | acc = 28.12%
Iter 010 | loss = 1.007585 | acc = 53.12%
Iter 020 | loss = 0.973565 | acc = 53.12%
Iter 030 | loss = 0.077861 | acc = 100.00%
Iter 040 | loss = 0.000408 | acc = 100.00%
Iter 050 | loss = 0.000021 | acc = 100.00%
Iter 060 | loss = 0.000009 | acc = 100.00%
Iter 070 | loss = 0.000005 | acc = 100.00%
Iter 080 | loss = 0.000003 | acc = 100.00%
Iter 090 | loss = 0.000002 | acc = 100.00%
Iter 100 | loss = 0.000002 | acc = 100.00%
Finished overfit test.


### Search the best Hyperparameter using Hold-out set

### Test with dropout rate = 0.1
- Transformer Model: Revised the model to add a dropout parameter in self-attention head and transformer block
- Change the Learning Rate Scheduler from LRstep to ReduceLROnPlateau for improving the validation loss decrease
- Step Size = 5, gamma = 0.5 -> If validation loss does not decrease in 5 epochs, the LR will be decreased by half

#### Implement Grid Search to find the search space

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Grid Search 후보값 ────────────────────────────────────────
LR_CHOICES          = [3e-4, 4e-4, 5e-4]
WD_CHOICES          = [5e-5, 5e-4, 1e-3]
NUM_FILTERS         = 120
NUM_BLOCK_CHOICES   = [2]
NUM_HEAD_CHOICES    = [3, 4]
SEGMENT_CHOICES     = [5]

# ─── Training configuration ───────────────────────────────────
MAX_EPOCHS  = 100
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ──────────────────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── Scheduler Hyperparameter ─────────────────────────────────
FIXED_STEP_SIZE = 5
FIXED_GAMMA     = 0.5


def objective_holdout(trial):
    # ─── 1) Grid용 sampling ───────────────────────────────────
    lr           = trial.suggest_categorical("lr", LR_CHOICES)
    weight_decay = trial.suggest_categorical("weight_decay", WD_CHOICES)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # ─── 2) 데이터 로드 ────────────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # ─── 3) Hold-out split ─────────────────────────────────────
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # ─── 4) W&B init ───────────────────────────────────────────
    wandb.init(
        project="eeg-holdout-grid-search",
        config={
            "lr": lr,
            "weight_decay": weight_decay,
            "num_blocks": num_blocks,
            "num_heads": num_heads,
            "num_segments": num_segments
        }
    )

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, "
        f"blocks={num_blocks}, heads={num_heads}, segs={num_segments}"
    )

    # ─── 5) Model / optimizer / loss ──────────────────────────
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    # ─── 6) Scheduler ──────────────────────────────────────────
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=FIXED_GAMMA,
        patience=FIXED_STEP_SIZE,
        min_lr=1e-6
    )

    # ─── 7) Training loop w/ Early Stopping & Pruning ─────────
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y    = X.to(DEVICE), y.to(DEVICE)
                logits  = model(X)
                loss    = criterion(logits, y)
                vloss   += loss.item()
                vcorrect+= (logits.argmax(1) == y).sum().item()
                vtotal  += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # — report & pruning check —
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — print & log —
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            "epoch":               epoch,
            "train_loss":          train_loss,
            "train_accuracy":      train_acc,
            "validation_loss":     val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        scheduler.step(val_loss)

        # — early stopping logic & save best metrics —
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()

    # ─── GridSampler용 파라미터 그리드 ─────────────────────────
    param_grid = {
        "lr":            LR_CHOICES,
        "weight_decay":  WD_CHOICES,
        "num_blocks":    NUM_BLOCK_CHOICES,
        "num_heads":     NUM_HEAD_CHOICES,
        "num_segments":  SEGMENT_CHOICES,
    }

    sampler = optuna.samplers.GridSampler(param_grid)
    study = optuna.create_study(
        direction="minimize",
        sampler=sampler,
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_grid_search",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_grid_search.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout)  # grid 크기만큼 자동 실행

    # ─── 결과 출력 ─────────────────────────────────────────────
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss       = {best.value:.6f}")
    print(f"best_train_loss     = {best.user_attrs['best_train_loss']:.6f}")
    print(f"best_train_accuracy = {best.user_attrs['best_train_acc']:.4f}")
    print(f"best_val_accuracy   = {best.user_attrs['best_val_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-02 00:26:22,491] A new study created in RDB with name: eeg_holdout_grid_search



===== Trial 0 =====
 lr=3.00e-04, wd=5.00e-04, blocks=2, heads=3, segs=5
Epoch 001 | train_loss=1.0704 acc=0.4237 | val_loss=1.0744 acc=0.4317 | time=30.6s
Epoch 002 | train_loss=1.0677 acc=0.4311 | val_loss=1.0756 acc=0.4317 | time=29.9s
Epoch 003 | train_loss=1.0672 acc=0.4311 | val_loss=1.0760 acc=0.4317 | time=29.9s
Epoch 004 | train_loss=1.0685 acc=0.4276 | val_loss=1.0817 acc=0.4317 | time=30.0s
Epoch 005 | train_loss=1.0665 acc=0.4260 | val_loss=1.0804 acc=0.4317 | time=29.9s
Epoch 006 | train_loss=1.0696 acc=0.4276 | val_loss=1.0754 acc=0.4317 | time=29.8s
Epoch 007 | train_loss=1.0673 acc=0.4311 | val_loss=1.0764 acc=0.4317 | time=30.0s
Epoch 008 | train_loss=1.0669 acc=0.4311 | val_loss=1.0747 acc=0.4317 | time=29.9s
Epoch 009 | train_loss=1.0666 acc=0.4311 | val_loss=1.0755 acc=0.4317 | time=30.0s
Epoch 010 | train_loss=1.0665 acc=0.4311 | val_loss=1.0749 acc=0.4317 | time=30.0s
Epoch 011 | train_loss=1.0671 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=30.0s
Epoch 012 | t

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁██▅▃▅███████████████████████████████
train_loss,█▄▃▅▃▇▄▃▃▃▃▃▃▂▁▂▃▂▂▂▂▃▁▃▂▃▂▂▃▂▂▂▂▂▁▂▂
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,▁▂▃█▇▂▃▁▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▂▁▂▂▂▂▁
epoch,37
train_accuracy,0.43107
train_loss,1.06584
validation_accuracy,0.43168
validation_loss,1.07483


[I 2025-05-02 00:44:55,381] Trial 0 finished with value: 1.0743041322344826 and parameters: {'lr': 0.0003, 'weight_decay': 0.0005, 'num_blocks': 2, 'num_heads': 3, 'num_segments': 5}. Best is trial 0 with value: 1.0743041322344826.


...

▸ Trial 16 pruned at epoch 8



===== Trial 17 =====
 lr=5.00e-04, wd=5.00e-05, blocks=2, heads=3, segs=5
Epoch 001 | train_loss=1.0704 acc=0.4311 | val_loss=1.0816 acc=0.4317 | time=30.2s
Epoch 002 | train_loss=1.0702 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=30.4s
Epoch 003 | train_loss=1.0670 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=30.1s
Epoch 004 | train_loss=1.0681 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=30.1s
Epoch 005 | train_loss=1.0677 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=30.1s
Epoch 006 | train_loss=1.0678 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=30.2s
Epoch 007 | train_loss=1.0667 acc=0.4311 | val_loss=1.0749 acc=0.4317 | time=30.1s


epoch,▁▂▃▅▆▇█
train_accuracy,▁▁▁▁▁▁▁
train_loss,█▇▁▃▃▃▁
validation_accuracy,▁▁▁▁▁▁▁
validation_loss,█▁▁▂▁▁▂
epoch,7
train_accuracy,0.43107
train_loss,1.06674
validation_accuracy,0.43168
validation_loss,1.07487


[I 2025-05-02 03:27:41,000] Trial 17 pruned. 


▸ Trial 17 pruned at epoch 8

===== Best Trial =====
best_val_loss       = 0.799622
best_train_loss     = 0.557883
best_train_accuracy = 0.7751
best_val_accuracy   = 0.6801
best params:
  lr: 0.0004
  weight_decay: 0.001
  num_blocks: 2
  num_heads: 3
  num_segments: 5


In [ ]:
===== Best Trial =====
best_val_loss       = 0.799622
best_train_loss     = 0.557883
best_train_accuracy = 0.7751
best_val_accuracy   = 0.6801
best params:
  lr: 0.0004
  weight_decay: 0.001
  num_blocks: 2
  num_heads: 3
  num_segments: 5

### Reduce the Model depth and width
- Decrease the model complexity due to overfitting
- Decrease the model blocks from 2 to 1
- Test it with different heads = [1,2,3]

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Grid Search 후보값 ────────────────────────────────────────
LR_CHOICES          = [3e-4, 4e-4, 5e-4]
WD_CHOICES          = [5e-5, 5e-4, 1e-3]
NUM_FILTERS         = 120
NUM_BLOCK_CHOICES   = [1]
NUM_HEAD_CHOICES    = [2, 3, 4]
SEGMENT_CHOICES     = [5]

# ─── Training configuration ───────────────────────────────────
MAX_EPOCHS  = 100
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ──────────────────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── Scheduler 하이퍼파라미터 ─────────────────────────────────
FIXED_STEP_SIZE = 5
FIXED_GAMMA     = 0.5


def objective_holdout(trial):
    # ─── 1) Grid용 sampling ───────────────────────────────────
    lr           = trial.suggest_categorical("lr", LR_CHOICES)
    weight_decay = trial.suggest_categorical("weight_decay", WD_CHOICES)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # ─── 2) 데이터 로드 ────────────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # ─── 3) Hold-out split ─────────────────────────────────────
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # ─── 4) W&B init ───────────────────────────────────────────
    wandb.init(
        project="eeg-holdout-grid-search",
        config={
            "lr": lr,
            "weight_decay": weight_decay,
            "num_blocks": num_blocks,
            "num_heads": num_heads,
            "num_segments": num_segments
        }
    )

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, "
        f"blocks={num_blocks}, heads={num_heads}, segs={num_segments}"
    )

    # ─── 5) Model / optimizer / loss ──────────────────────────
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    # ─── 6) Scheduler ──────────────────────────────────────────
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=FIXED_GAMMA,
        patience=FIXED_STEP_SIZE,
        min_lr=1e-6
    )

    # ─── 7) Training loop w/ Early Stopping & Pruning ─────────
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y    = X.to(DEVICE), y.to(DEVICE)
                logits  = model(X)
                loss    = criterion(logits, y)
                vloss   += loss.item()
                vcorrect+= (logits.argmax(1) == y).sum().item()
                vtotal  += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # — report & pruning check —
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — print & log —
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            "epoch":               epoch,
            "train_loss":          train_loss,
            "train_accuracy":      train_acc,
            "validation_loss":     val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        scheduler.step(val_loss)

        # — early stopping logic & save best metrics —
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()

    # ─── GridSampler용 파라미터 그리드 ─────────────────────────
    param_grid = {
        "lr":            LR_CHOICES,
        "weight_decay":  WD_CHOICES,
        "num_blocks":    NUM_BLOCK_CHOICES,
        "num_heads":     NUM_HEAD_CHOICES,
        "num_segments":  SEGMENT_CHOICES,
    }

    sampler = optuna.samplers.GridSampler(param_grid)
    study = optuna.create_study(
        direction="minimize",
        sampler=sampler,
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_grid_search",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_grid_search.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout)  # grid 크기만큼 자동 실행

    # ─── 결과 출력 ─────────────────────────────────────────────
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss       = {best.value:.6f}")
    print(f"best_train_loss     = {best.user_attrs['best_train_loss']:.6f}")
    print(f"best_train_accuracy = {best.user_attrs['best_train_acc']:.4f}")
    print(f"best_val_accuracy   = {best.user_attrs['best_val_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-4, 1e-2
WD_MIN, WD_MAX     = 1e-6, 1e-3

NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [2, 3]
NUM_HEAD_CHOICES   = [2, 4]
SEGMENT_CHOICES    = [5]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 100   # 100 epochs 고정
PATIENCE    = 20    # Early stopping patience
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ─── 고정 스케줄러 하이퍼파라미터 ─────────────────────────
FIXED_STEP_SIZE = 5
FIXED_GAMMA     = 0.5


def objective_holdout(trial):
    # ─ 1) Sample hyperparameters ────────────────────────────────
    lr           = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # ─ 2) Load data ────────────────────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # ─ 3) Hold-out split ──────────────────────────────────────────
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # ─ 4) W&B init ────────────────────────────────────────────────
    wandb.init(project="eeg-holdout-tuning-9", config=trial.params)

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, "
        f"blocks={num_blocks}, heads={num_heads}, segs={num_segments}")

    # ─ 5) Model / optimizer / loss ───────────────────────────────
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay  # L2
    )
    criterion = nn.CrossEntropyLoss()

    # ─ 6) Scheduler ───────────────────────────────────────────────
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=FIXED_GAMMA,
        patience=FIXED_STEP_SIZE,
        min_lr=1e-6
    )

    # ─ 7) Training loop w/ Early Stopping & Pruning ──────────────
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y    = X.to(DEVICE), y.to(DEVICE)
                logits  = model(X)
                loss    = criterion(logits, y)
                vloss   += loss.item()
                vcorrect+= (logits.argmax(1) == y).sum().item()
                vtotal  += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # — report & pruning check —
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — print & log —
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s | "
        )
        wandb.log({
            "epoch":               epoch,
            "train_loss":          train_loss,
            "train_accuracy":      train_acc,
            "validation_loss":     val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        scheduler.step(val_loss)

        # — early stopping logic & save best metrics —
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial-9",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout-9.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=20)

    # ─── 결과 출력 ────────────────────────────────────────────────
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss       = {best.value:.6f}")
    print(f"best_train_loss     = {best.user_attrs['best_train_loss']:.6f}")
    print(f"best_train_accuracy = {best.user_attrs['best_train_acc']:.4f}")
    print(f"best_val_accuracy   = {best.user_attrs['best_val_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-05-01 22:22:27,459] Using an existing study with name 'eeg_holdout_trial-9' instead of creating a new one.



===== Trial 1 =====
 lr=2.35e-04, wd=7.48e-05, blocks=2, heads=4, segs=5
Epoch 001 | train_loss=1.0738 acc=0.4214 | val_loss=1.0746 acc=0.4317 | time=36.7s | 
Epoch 002 | train_loss=1.0684 acc=0.4311 | val_loss=1.0751 acc=0.4317 | time=36.0s | 
Epoch 003 | train_loss=1.0684 acc=0.4311 | val_loss=1.0751 acc=0.4317 | time=36.3s | 
Epoch 004 | train_loss=1.0671 acc=0.4311 | val_loss=1.0749 acc=0.4317 | time=36.3s | 
Epoch 005 | train_loss=1.0675 acc=0.4311 | val_loss=1.0748 acc=0.4317 | time=36.1s | 
Epoch 006 | train_loss=1.0661 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=36.1s | 
Epoch 007 | train_loss=1.0669 acc=0.4311 | val_loss=1.0764 acc=0.4317 | time=36.0s | 
Epoch 008 | train_loss=1.0677 acc=0.4311 | val_loss=1.0752 acc=0.4317 | time=36.1s | 
Epoch 009 | train_loss=1.0667 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=36.2s | 
Epoch 010 | train_loss=1.0671 acc=0.4311 | val_loss=1.0763 acc=0.4317 | time=36.3s | 
Epoch 011 | train_loss=1.0677 acc=0.4311 | val_loss=1.0750 acc=0.4

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇███████
train_loss,█████████████▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▆▆▆▆▆▆▆▆▆▇▆█████████▇█
validation_loss,██████████████▆▅▄▄▃▃▂▂▃▂▂▂▂▁▂▂▄▁▂▁▁▂▂▃▃▃
epoch,77
train_accuracy,0.7965
train_loss,0.51931
validation_accuracy,0.65839
validation_loss,0.93526


[I 2025-05-01 23:08:57,626] Trial 1 finished with value: 0.8704692948432196 and parameters: {'lr': 0.00023450279490485625, 'weight_decay': 7.480682204115078e-05, 'num_blocks': 2, 'num_heads': 4, 'num_segments': 5}. Best is trial 1 with value: 0.8704692948432196.


...

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_accuracy,▁▁▁▁▁▂▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_loss,██████▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁▁
validation_accuracy,▁▁▁▁▁▆▇▆▇█▇▇▇▇▇▆▇▅▇▇▇█▇███▇███
validation_loss,▄▄▄▄▄▃▂▃▁▁▁▁▂▂▂▂▂▃▂▃▃▁▄▃▄▅▅▅▆█
epoch,30
train_accuracy,0.86951
train_loss,0.35468
validation_accuracy,0.61491
validation_loss,1.35241


[I 2025-05-01 23:21:21,543] Trial 2 finished with value: 0.9047478267124721 and parameters: {'lr': 0.0006332497165117276, 'weight_decay': 6.1060665269356535e-06, 'num_blocks': 2, 'num_heads': 2, 'num_segments': 5}. Best is trial 1 with value: 0.8704692948432196.



===== Trial 3 =====
 lr=5.07e-03, wd=1.51e-05, blocks=3, heads=4, segs=5


[W 2025-05-01 23:21:28,840] Trial 3 failed with parameters: {'lr': 0.005073462485729068, 'weight_decay': 1.5078657828589909e-05, 'num_blocks': 3, 'num_heads': 4, 'num_segments': 5} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 2.48 GiB. GPU 0 has a total capacity of 39.56 GiB of which 2.06 GiB is free. Process 734759 has 37.49 GiB memory in use. Of the allocated memory 35.18 GiB is allocated by PyTorch, and 1.80 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-8-173f8f6107e4>", line 126

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.48 GiB. GPU 0 has a total capacity of 39.56 GiB of which 2.06 GiB is free. Process 734759 has 37.49 GiB memory in use. Of the allocated memory 35.18 GiB is allocated by PyTorch, and 1.80 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Test with dropout = 0.3
- Transformer Model: Revised the model to add a dropout parameter in self-attention head and transformer block
- Change the Learning Rate Scheduler from LRstep to ReduceLROnPlateau for improving the validation loss decrease

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-5, 5e-3
WD_MIN, WD_MAX     = 1e-6, 1e-3
L1_MIN, L1_MAX     = 1e-7, 1e-3

NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [1, 2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]
SEGMENT_CHOICES    = [5, 10, 15]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 100   # 100 epochs 고정
PATIENCE    = 20    # Early stopping patience
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective_holdout(trial):
    # ─ 1) Sample hyperparameters ────────────────────────────────
    lr           = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    use_l1       = trial.suggest_categorical("use_l1", [False, True])
    l1_lambda    = trial.suggest_float("l1_lambda", L1_MIN, L1_MAX, log=True) if use_l1 else 0.0

    step_size    = trial.suggest_int("step_size", 10, 20, step=5)
    gamma        = trial.suggest_float("gamma", 0.1, 0.9)

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, "
        f"L1={'on' if use_l1 else 'off'}{f'({l1_lambda:.2e})' if use_l1 else ''}, "
        f"blocks={num_blocks}, heads={num_heads}, segs={num_segments}, "
        f"step_size={step_size}, gamma={gamma:.2f}"
    )

    # ─ 2) Load data ────────────────────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # ─ 3) Hold-out split ──────────────────────────────────────────
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # ─ 4) W&B init ────────────────────────────────────────────────
    wandb.init(project="eeg-holdout-tuning-4", config=trial.params)

    # ─ 5) Model / optimizer / loss ───────────────────────────────
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay  # L2
    )
    criterion = nn.CrossEntropyLoss()

    # ─ 6) Scheduler ───────────────────────────────────────────────
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                           factor=gamma, patience=step_size,
                                                           min_lr=1e-6)

    # ─ 7) Training loop w/ Early Stopping & Pruning ──────────────
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    # placeholders for best-epoch metrics
    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss    = loss + l1_lambda * l1_norm
            loss.backward()
            optimizer.step()
            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss    += loss.item()
                vcorrect += (logits.argmax(1) == y).sum().item()
                vtotal   += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # — report & pruning check —
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — print & log —
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            "epoch":               epoch,
            "train_loss":          train_loss,
            "train_accuracy":      train_acc,
            "validation_loss":     val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        scheduler.step(val_loss)

        # — early stopping logic & save best metrics —
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial-4",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout-4.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=20)

    # ─── 결과 출력 ────────────────────────────────────────────────
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss       = {best.value:.6f}")
    print(f"best_train_loss     = {best.user_attrs['best_train_loss']:.6f}")
    print(f"best_train_accuracy = {best.user_attrs['best_train_acc']:.4f}")
    print(f"best_val_accuracy   = {best.user_attrs['best_val_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


[I 2025-05-01 10:20:21,793] A new study created in RDB with name: eeg_holdout_trial-4



===== Trial 0 =====
 lr=6.47e-05, wd=3.44e-05, L1=on(3.29e-05), blocks=1, heads=3, segs=5, step_size=15, gamma=0.72


Epoch 001 | train_loss=17.0697 acc=0.4206 | val_loss=1.0745 acc=0.4317 | time=223.0s
Epoch 002 | train_loss=16.8072 acc=0.4311 | val_loss=1.0747 acc=0.4317 | time=18.7s
Epoch 003 | train_loss=16.5682 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=18.7s
Epoch 004 | train_loss=16.3461 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=18.7s
Epoch 005 | train_loss=16.1411 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=18.6s
Epoch 006 | train_loss=15.9511 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=18.8s
Epoch 007 | train_loss=15.7722 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=18.6s
Epoch 008 | train_loss=15.6012 acc=0.4311 | val_loss=1.0749 acc=0.4317 | time=18.8s
Epoch 009 | train_loss=15.4385 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=18.5s
Epoch 010 | train_loss=15.2845 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=18.6s
Epoch 011 | train_loss=15.1332 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=18.5s
Epoch 012 | train_loss=14.9882 acc=0.4311 | val_loss=1.0749 acc=0.4317 | ti

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
train_accuracy,▁██████████████████████
train_loss,█▇▇▆▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,▂▃▁▂▇▁▂▆▇▃▃▅▅▇▆▇▂▄▅▄▄▅█
epoch,23
train_accuracy,0.43107
train_loss,13.85227
validation_accuracy,0.43168
validation_loss,1.07512


[I 2025-05-01 10:30:58,770] Trial 0 finished with value: 1.074448063260033 and parameters: {'lr': 6.469169596116346e-05, 'weight_decay': 3.4401973392369906e-05, 'num_blocks': 1, 'num_heads': 3, 'num_segments': 5, 'use_l1': True, 'l1_lambda': 3.2893090509651604e-05, 'step_size': 15, 'gamma': 0.7165871970832125}. Best is trial 0 with value: 1.074448063260033.


...

[I 2025-05-01 11:34:46,786] Trial 4 finished with value: 1.0742088499523343 and parameters: {'lr': 0.00421821567844694, 'weight_decay': 0.00011946286871298436, 'num_blocks': 2, 'num_heads': 3, 'num_segments': 10, 'use_l1': False, 'step_size': 20, 'gamma': 0.26986870444488426}. Best is trial 2 with value: 1.0742079076312838.



===== Trial 5 =====
 lr=1.41e-05, wd=4.77e-06, L1=on(2.59e-05), blocks=2, heads=4, segs=10, step_size=20, gamma=0.67


Epoch 001 | train_loss=14.3605 acc=0.2975 | val_loss=1.0913 acc=0.4317 | time=36.8s
Epoch 002 | train_loss=14.2887 acc=0.4311 | val_loss=1.0802 acc=0.4317 | time=36.7s
Epoch 003 | train_loss=14.2266 acc=0.4311 | val_loss=1.0761 acc=0.4317 | time=37.0s
Epoch 004 | train_loss=14.1707 acc=0.4311 | val_loss=1.0748 acc=0.4317 | time=36.8s


[W 2025-05-01 11:37:44,572] Trial 5 failed with parameters: {'lr': 1.4148595050869106e-05, 'weight_decay': 4.7749477404437315e-06, 'num_blocks': 2, 'num_heads': 4, 'num_segments': 10, 'use_l1': True, 'l1_lambda': 2.5920124189920035e-05, 'step_size': 20, 'gamma': 0.6651222578443667} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-9-e3aa22c865ab>", line 136, in objective_holdout
    tloss    += loss.item()
                ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-05-01 11:37:44,574] Trial 5 failed with value None.


KeyboardInterrupt: 

### Test with dropout = 0.2 Model
- Cause Model Overfitting
- Validation Loss fastly decreases and increases after that
- Validation Accuracy increases, but the Validation Loss does not improve after some amount of epochs
- This is due to the overfitting and over confidence that model predicts the correct label as answer but cannot sure about it's answer

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-5, 5e-3
WD_MIN, WD_MAX     = 1e-6, 1e-3
L1_MIN, L1_MAX     = 1e-7, 1e-3

NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [1, 2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]
SEGMENT_CHOICES    = [5, 10, 15]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 100   # 100 epochs 고정
PATIENCE    = 20    # Early stopping patience
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective_holdout(trial):
    # ─ 1) Sample hyperparameters ────────────────────────────────
    lr           = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    use_l1       = trial.suggest_categorical("use_l1", [False, True])
    l1_lambda    = trial.suggest_float("l1_lambda", L1_MIN, L1_MAX, log=True) if use_l1 else 0.0

    step_size    = trial.suggest_int("step_size", 10, 30, step=10)
    gamma        = trial.suggest_float("gamma", 0.1, 0.9)

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, "
        f"L1={'on' if use_l1 else 'off'}{f'({l1_lambda:.2e})' if use_l1 else ''}, "
        f"blocks={num_blocks}, heads={num_heads}, segs={num_segments}, "
        f"step_size={step_size}, gamma={gamma:.2f}"
    )

    # ─ 2) Load data ────────────────────────────────────────────────
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # ─ 3) Hold-out split ──────────────────────────────────────────
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # ─ 4) W&B init ────────────────────────────────────────────────
    wandb.init(project="eeg-holdout-tuning-3", config=trial.params)

    # ─ 5) Model / optimizer / loss ───────────────────────────────
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay  # L2
    )
    criterion = nn.CrossEntropyLoss()

    # ─ 6) Scheduler ───────────────────────────────────────────────
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=step_size,
        gamma=gamma
    )

    # ─ 7) Training loop w/ Early Stopping & Pruning ──────────────
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    # placeholders for best-epoch metrics
    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            if l1_lambda > 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss    = loss + l1_lambda * l1_norm
            loss.backward()
            optimizer.step()
            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss    += loss.item()
                vcorrect += (logits.argmax(1) == y).sum().item()
                vtotal   += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # — report & pruning check —
        trial.report(val_loss, epoch)
        if trial.should_prune():
            wandb.finish()
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # — print & log —
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )
        wandb.log({
            "epoch":               epoch,
            "train_loss":          train_loss,
            "train_accuracy":      train_acc,
            "validation_loss":     val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        scheduler.step()

        # — early stopping logic & save best metrics —
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial-3",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout-3.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=30)

    # ─── 결과 출력 ────────────────────────────────────────────────
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss       = {best.value:.6f}")
    print(f"best_train_loss     = {best.user_attrs['best_train_loss']:.6f}")
    print(f"best_train_accuracy = {best.user_attrs['best_train_acc']:.4f}")
    print(f"best_val_accuracy   = {best.user_attrs['best_val_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


[I 2025-05-01 05:26:38,791] A new study created in RDB with name: eeg_holdout_trial-3



===== Trial 0 =====
 lr=3.77e-03, wd=2.27e-06, L1=off, blocks=3, heads=3, segs=15, step_size=20, gamma=0.39


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,██▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▃▃▂▄▂▁▃▁▂▂▁▁▁▂▂▁▂▁▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▁▁▂
epoch,50
train_accuracy,0.43107
train_loss,3.89059
validation_accuracy,0.43168
validation_loss,1.0748


Epoch 001 | train_loss=1.0715 acc=0.4291 | val_loss=1.0786 acc=0.4317 | time=38.8s
Epoch 002 | train_loss=1.0694 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=38.6s
Epoch 003 | train_loss=1.0699 acc=0.4260 | val_loss=1.0767 acc=0.4317 | time=38.7s
Epoch 004 | train_loss=1.0687 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=38.5s
Epoch 005 | train_loss=1.0678 acc=0.4221 | val_loss=1.0767 acc=0.4317 | time=38.7s
Epoch 006 | train_loss=1.0675 acc=0.4311 | val_loss=1.0764 acc=0.4317 | time=38.6s
Epoch 007 | train_loss=1.0685 acc=0.4311 | val_loss=1.0752 acc=0.4317 | time=38.7s
Epoch 008 | train_loss=1.0674 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=38.7s
Epoch 009 | train_loss=1.0675 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=38.8s
Epoch 010 | train_loss=1.0671 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=38.8s
Epoch 011 | train_loss=1.0670 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=38.8s
Epoch 012 | train_loss=1.0665 acc=0.4311 | val_loss=1.0747 acc=0.4317 | time=38.7s
Epoc

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train_accuracy,▆█▄█▁█████████████████████████████
train_loss,█▆▆▅▄▃▄▃▃▃▃▂▂▃▂▂▄▄▂▃▂▂▃▁▁▂▂▁▂▂▂▁▂▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▂▅▁▅▅▃▂▂▁▁▂▃▁▂▁▂▂▂▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂
epoch,34
train_accuracy,0.43107
train_loss,1.06561
validation_accuracy,0.43168
validation_loss,1.0747


...

Epoch 001 | train_loss=1.0709 acc=0.4245 | val_loss=1.0811 acc=0.4317 | time=20.4s
Epoch 002 | train_loss=1.0691 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=20.3s
Epoch 003 | train_loss=1.0703 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=20.5s
Epoch 004 | train_loss=1.0680 acc=0.4311 | val_loss=1.0754 acc=0.4317 | time=20.3s
Epoch 005 | train_loss=1.0683 acc=0.4311 | val_loss=1.0834 acc=0.4317 | time=20.3s
Epoch 006 | train_loss=1.0688 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=20.4s
Epoch 007 | train_loss=1.0681 acc=0.4311 | val_loss=1.0772 acc=0.4317 | time=20.3s
Epoch 008 | train_loss=1.0676 acc=0.4311 | val_loss=1.0752 acc=0.4317 | time=20.4s
Epoch 009 | train_loss=1.0668 acc=0.4311 | val_loss=1.0753 acc=0.4317 | time=20.3s
Epoch 010 | train_loss=1.0675 acc=0.4311 | val_loss=1.0749 acc=0.4317 | time=20.3s
Epoch 011 | train_loss=1.0658 acc=0.4311 | val_loss=1.0711 acc=0.4317 | time=20.4s
Epoch 012 | train_loss=1.0330 acc=0.4924 | val_loss=0.9821 acc=0.5295 | time=20.2s
Epoc

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
train_accuracy,▁▁▁▁▁▁▁▁▁▂▄▄▄▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇█▇██████████
train_loss,█████████▇▇▆▆▆▆▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▄▅▅▅▆▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇██████████
validation_loss,██████▅▅▅▄▄▃▃▃▃▃▃▂▃▂▂▃▃▃▄▁▂▄▄▄▅▄▄▄▅▅▅▅▆▆
epoch,58
train_accuracy,0.86796
train_loss,0.33339
validation_accuracy,0.67391
validation_loss,0.99908


[I 2025-05-01 09:38:43,220] Trial 17 finished with value: 0.8123467592965989 and parameters: {'lr': 0.0006613322502150471, 'weight_decay': 5.306026913460125e-05, 'num_blocks': 1, 'num_heads': 4, 'num_segments': 5, 'use_l1': False, 'step_size': 20, 'gamma': 0.20561411467734086}. Best is trial 11 with value: 0.6841289514587039.



===== Trial 18 =====
 lr=1.90e-04, wd=1.72e-04, L1=on(1.48e-05), blocks=1, heads=2, segs=10, step_size=10, gamma=0.39


Epoch 001 | train_loss=8.1850 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=17.3s
Epoch 002 | train_loss=7.9208 acc=0.4311 | val_loss=1.0747 acc=0.4317 | time=17.3s
Epoch 003 | train_loss=7.6896 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=17.5s
Epoch 004 | train_loss=7.4814 acc=0.4311 | val_loss=1.0813 acc=0.4317 | time=17.5s
Epoch 005 | train_loss=7.2928 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=17.6s
Epoch 006 | train_loss=7.1234 acc=0.4311 | val_loss=1.0748 acc=0.4317 | time=17.5s
Epoch 007 | train_loss=6.9673 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=17.3s
Epoch 008 | train_loss=6.8240 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=17.6s
Epoch 009 | train_loss=6.6866 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=17.5s


In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-4, 1e-3
WD_MIN, WD_MAX     = 1e-5, 1e-3
NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [1, 2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]
SEGMENT_CHOICES    = [5, 10, 15]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 200
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective_holdout(trial):
    # 1) Sample hyperparameters
    lr            = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay  = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks    = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads     = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments  = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)
    # New: sample scheduler hyperparameters
    step_size     = trial.suggest_int("step_size", 10, 30, step=10)
    gamma         = trial.suggest_float("gamma", 0.1, 0.9)

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, blocks={num_blocks}, "
        f"heads={num_heads}, segs={num_segments}, "
        f"step_size={step_size}, gamma={gamma:.2f}"
    )

    # 2) Load metadata and build dataset
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta  = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, train_meta)
    labels      = [d["label"] for d in train_meta]
    n_samples   = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # 3) Single hold-out split (80/20), stratified
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # 4) W&B init
    wandb.init(project="eeg-holdout-tuning-1", config=trial.params)

    # 5) Build model, optimizer, loss
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    # 6) Scheduler with sampled step_size & gamma
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=step_size,
        gamma=gamma
    )

    # 7) Training loop with early stopping & pruning
    best_val_loss     = float("inf")
    epochs_no_improve = 0
    # store best-epoch metrics
    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss    += loss.item()
                vcorrect += (logits.argmax(1) == y).sum().item()
                vtotal   += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # report & prune
        trial.report(val_loss, epoch)
        if trial.should_prune():
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )

        # — log all four metrics to wandb for graphing
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "validation_loss": val_loss,
            "validation_accuracy": val_acc,
        }, step=epoch)

        # step the scheduler
        scheduler.step()

        # early stopping & record best
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best-epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial-1",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout-1.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=30)

    # print best trial & metrics
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss   = {best.value:.6f}")
    print(f"best_val_acc    = {best.user_attrs['best_val_acc']:.4f}")
    print(f"best_train_loss = {best.user_attrs['best_train_loss']:.4f}")
    print(f"best_train_acc  = {best.user_attrs['best_train_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


[I 2025-04-30 22:40:14,173] Using an existing study with name 'eeg_holdout_trial-1' instead of creating a new one.



===== Trial 1 =====
 lr=2.50e-04, wd=1.45e-04, blocks=1, heads=4, segs=10, step_size=20, gamma=0.70


epoch,▁█
train_accuracy,▁█
train_loss,▁█
validation_accuracy,▁█
validation_loss,█▁
epoch,2
train_accuracy,0.42951
train_loss,1.07145
validation_accuracy,0.43168
validation_loss,1.07453


Epoch 001 | train_loss=1.0706 acc=0.4260 | val_loss=1.0761 acc=0.4317 | time=20.7s
Epoch 002 | train_loss=1.0684 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=20.4s
Epoch 003 | train_loss=1.0676 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=20.3s
Epoch 004 | train_loss=1.0671 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=20.5s
Epoch 005 | train_loss=1.0679 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=20.4s
Epoch 006 | train_loss=1.0684 acc=0.4311 | val_loss=1.0762 acc=0.4317 | time=20.5s
Epoch 007 | train_loss=1.0676 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=20.4s
Epoch 008 | train_loss=1.0669 acc=0.4311 | val_loss=1.0765 acc=0.4317 | time=20.5s
Epoch 009 | train_loss=1.0672 acc=0.4311 | val_loss=1.0836 acc=0.4317 | time=20.4s
Epoch 010 | train_loss=1.0709 acc=0.4249 | val_loss=1.0759 acc=0.4317 | time=20.4s
Epoch 011 | train_loss=1.0673 acc=0.4311 | val_loss=1.0768 acc=0.4317 | time=20.6s
Epoch 012 | train_loss=1.0681 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=20.3s
Epoc

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇▇█████████
train_loss,█████████████▇▇▇▇▆▆▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█▇█▇▇█▇█
validation_loss,▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▃▂▂▃▅▃▄▅▇▆▇█▇
epoch,49
train_accuracy,0.97165
train_loss,0.09872
validation_accuracy,0.67857
validation_loss,2.05648


...

▸ Trial 29 pruned at epoch 20

===== Trial 30 =====
 lr=3.14e-04, wd=1.66e-04, blocks=2, heads=3, segs=10, step_size=20, gamma=0.66


epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
train_accuracy,▁▁▁▁▁▁▁▃▅▆▆▆▆▇▇▆▇▇█
train_loss,███████▇▆▅▅▄▄▃▃▃▂▂▁
validation_accuracy,▁▁▁▁▁▁▁▅▆▆▆▆▇█▇▆█▇█
validation_loss,███████▇▃▂▄▄▁▄▁▂▂▂▂
epoch,19
train_accuracy,0.69126
train_loss,0.72615
validation_accuracy,0.62112
validation_loss,0.94252


Epoch 001 | train_loss=1.0702 acc=0.4249 | val_loss=1.0793 acc=0.3416 | time=28.4s
Epoch 002 | train_loss=1.0696 acc=0.4237 | val_loss=1.0742 acc=0.4317 | time=28.6s
Epoch 003 | train_loss=1.0681 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=28.5s
Epoch 004 | train_loss=1.0690 acc=0.4311 | val_loss=1.0745 acc=0.4317 | time=28.7s
Epoch 005 | train_loss=1.0667 acc=0.4311 | val_loss=1.0751 acc=0.4317 | time=28.3s
Epoch 006 | train_loss=1.0673 acc=0.4311 | val_loss=1.0757 acc=0.4317 | time=28.2s
Epoch 007 | train_loss=1.0668 acc=0.4311 | val_loss=1.0766 acc=0.4317 | time=28.3s
Epoch 008 | train_loss=1.0668 acc=0.4311 | val_loss=1.0720 acc=0.4317 | time=28.5s
Epoch 009 | train_loss=1.0378 acc=0.4819 | val_loss=0.9859 acc=0.5543 | time=28.4s
Epoch 010 | train_loss=0.9629 acc=0.5682 | val_loss=0.9659 acc=0.5714 | time=28.3s
Epoch 011 | train_loss=0.9239 acc=0.5988 | val_loss=0.9472 acc=0.5854 | time=28.4s
Epoch 012 | train_loss=0.8993 acc=0.6027 | val_loss=0.9550 acc=0.5870 | time=28.5s
Epoc

[I 2025-05-01 02:35:40,692] Trial 30 pruned. 


▸ Trial 30 pruned at epoch 20

===== Best Trial =====
best_val_loss   = 0.575340
best_val_acc    = 0.7671
best_train_loss = 0.3076
best_train_acc  = 0.8816
best params:
  lr: 0.0009667592898435953
  weight_decay: 9.995420002148595e-05
  num_blocks: 3
  num_heads: 2
  num_segments: 5
  step_size: 30
  gamma: 0.4573463771689297


In [ ]:
print("hello")

hello


In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-4, 1e-3
WD_MIN, WD_MAX     = 1e-5, 1e-3
NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [1, 2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]
SEGMENT_CHOICES    = [5, 10, 15]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 200
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective_holdout(trial):
    # 1) Sample hyperparameters
    lr           = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    print(f"\n===== Trial {trial.number} =====")
    print(f" lr={lr:.2e}, wd={weight_decay:.2e}, blocks={num_blocks}, "
          f"heads={num_heads}, segs={num_segments}")

    # 2) 데이터 로드
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # 3) 한 번의 Hold-out split (80/20)
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # 4) W&B init (optional)
    wandb.init(project="eeg-holdout-tuning", config=trial.params)

    # 5) Model / Optimizer / Loss
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    # 6) Train with early-stopping & pruning
    best_val_loss     = float("inf")
    epochs_no_improve = 0

    # **여기에 저장용 변수를 선언**
    best_train_loss = None
    best_train_acc  = None
    best_val_acc    = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()
        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward(); optimizer.step()
            tloss     += loss.item()
            tcorrect  += (logits.argmax(1) == y).sum().item()
            ttotal    += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — val —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss    += loss.item()
                vcorrect += (logits.argmax(1) == y).sum().item()
                vtotal   += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # pruning
        trial.report(val_loss, epoch)
        if trial.should_prune():
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        print(f"Epoch {epoch:03d} | "
              f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
              f"time={elapsed:.1f}s")

        # early stopping & **최적시 train/val 지표 저장**
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # **최적 epoch 시 지표들을 user_attrs 에 저장**
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=30)

    # — 최종적으로 best trial 의 값과 user_attrs 를 함께 출력 —
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss   = {best.value:.6f}")
    print(f"best_val_acc    = {best.user_attrs['best_val_acc']:.4f}")
    print(f"best_train_loss = {best.user_attrs['best_train_loss']:.4f}")
    print(f"best_train_acc  = {best.user_attrs['best_train_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-04-30 19:01:36,475] A new study created in RDB with name: eeg_holdout_trial



===== Trial 0 =====
 lr=1.10e-04, wd=1.46e-04, blocks=1, heads=2, segs=10


Epoch 001 | train_loss=1.0711 acc=0.4311 | val_loss=1.0785 acc=0.4317 | time=330.3s
Epoch 002 | train_loss=1.0664 acc=0.4311 | val_loss=1.0804 acc=0.4317 | time=17.3s
Epoch 003 | train_loss=1.0679 acc=0.4311 | val_loss=1.0751 acc=0.4317 | time=17.3s
Epoch 004 | train_loss=1.0668 acc=0.4311 | val_loss=1.0756 acc=0.4317 | time=17.3s
Epoch 005 | train_loss=1.0685 acc=0.4311 | val_loss=1.0742 acc=0.4317 | time=17.1s
Epoch 006 | train_loss=1.0673 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=17.5s
Epoch 007 | train_loss=1.0673 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=17.5s
Epoch 008 | train_loss=1.0668 acc=0.4311 | val_loss=1.0770 acc=0.4317 | time=17.4s
Epoch 009 | train_loss=1.0669 acc=0.4311 | val_loss=1.0750 acc=0.4317 | time=17.4s
Epoch 010 | train_loss=1.0681 acc=0.4311 | val_loss=1.0746 acc=0.4317 | time=17.2s
Epoch 011 | train_loss=1.0662 acc=0.4311 | val_loss=1.0743 acc=0.4317 | time=17.2s
Epoch 012 | train_loss=1.0667 acc=0.4311 | val_loss=1.0754 acc=0.4317 | time=17.1s
Epo

[I 2025-04-30 19:22:37,535] Trial 0 finished with value: 0.9102347521554857 and parameters: {'lr': 0.00011016247004786675, 'weight_decay': 0.00014633689947303344, 'num_blocks': 1, 'num_heads': 2, 'num_segments': 10}. Best is trial 0 with value: 0.9102347521554857.


...

[I 2025-04-30 22:34:44,697] Trial 7 finished with value: 0.7561915225925899 and parameters: {'lr': 0.00020419417762750276, 'weight_decay': 0.00017949537675946488, 'num_blocks': 3, 'num_heads': 4, 'num_segments': 5}. Best is trial 2 with value: 0.7152097299695015.



===== Trial 8 =====
 lr=2.63e-04, wd=6.35e-04, blocks=2, heads=3, segs=5


Epoch 001 | train_loss=1.0713 acc=0.4311 | val_loss=1.0744 acc=0.4317 | time=27.8s
Epoch 002 | train_loss=1.0657 acc=0.4311 | val_loss=1.0765 acc=0.4317 | time=28.2s
Epoch 003 | train_loss=1.0693 acc=0.4311 | val_loss=1.0759 acc=0.4317 | time=27.8s
Epoch 004 | train_loss=1.0677 acc=0.4311 | val_loss=1.0752 acc=0.4317 | time=27.7s
Epoch 005 | train_loss=1.0678 acc=0.4311 | val_loss=1.0753 acc=0.4317 | time=27.8s


[W 2025-04-30 22:37:25,936] Trial 8 failed with parameters: {'lr': 0.0002627823846413076, 'weight_decay': 0.0006345325565238551, 'num_blocks': 2, 'num_heads': 3, 'num_segments': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-8-f3a55d92be6f>", line 111, in objective_holdout
    tloss     += loss.item()
                 ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-30 22:37:25,938] Trial 8 failed with value None.


KeyboardInterrupt: 

### Search the best Hyperparameter using 5-Fold Cross-Validation

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-4, 1e-3
WD_MIN, WD_MAX     = 1e-5, 1e-3
NUM_FILTERS        = 120
NUM_BLOCK_CHOICES  = [1, 2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]
SEGMENT_CHOICES    = [5, 10, 15]

# ─── Training configuration ─────────────────────────
MAX_EPOCHS  = 200
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective_holdout(trial):
    # 1) Sample hyperparameters
    lr            = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay  = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_blocks    = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads     = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments  = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)
    # New: sample scheduler hyperparameters
    step_size     = trial.suggest_int("step_size", 10, 100, step=10)
    gamma         = trial.suggest_float("gamma", 0.1, 0.9)

    print(f"\n===== Trial {trial.number} =====")
    print(
        f" lr={lr:.2e}, wd={weight_decay:.2e}, blocks={num_blocks}, "
        f"heads={num_heads}, segs={num_segments}, "
        f"step_size={step_size}, gamma={gamma:.2f}"
    )

    # 2) Load metadata and build dataset
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta  = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, train_meta)
    labels      = [d["label"] for d in train_meta]
    n_samples   = len(full_ds)
    input_length = full_ds[0][0].shape[-1]

    # 3) Single hold-out split (80/20), stratified
    train_idx, val_idx = train_test_split(
        list(range(n_samples)),
        test_size=0.2,
        stratify=labels,
        random_state=42
    )
    train_loader = DataLoader(
        Subset(full_ds, train_idx),
        batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS
    )
    val_loader = DataLoader(
        Subset(full_ds, val_idx),
        batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS
    )

    # 4) W&B init
    wandb.init(project="eeg-holdout-tuning", config=trial.params)

    # 5) Build model, optimizer, loss
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = num_heads,
        num_blocks   = num_blocks,
        num_segments = num_segments,
        num_classes  = 3
    ).to(DEVICE)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )
    criterion = nn.CrossEntropyLoss()

    # 6) Scheduler with sampled step_size & gamma
    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=step_size,
        gamma=gamma
    )

    # 7) Training loop with early stopping & pruning
    best_val_loss     = float("inf")
    epochs_no_improve = 0
    # store best‐epoch metrics
    best_train_loss = best_train_acc = best_val_acc = None

    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()

        # — train —
        model.train()
        tloss = tcorrect = ttotal = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()
            tloss    += loss.item()
            tcorrect += (logits.argmax(1) == y).sum().item()
            ttotal   += y.size(0)
        train_loss = tloss / len(train_loader)
        train_acc  = tcorrect / ttotal

        # — validate —
        model.eval()
        vloss = vcorrect = vtotal = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                logits = model(X)
                loss   = criterion(logits, y)
                vloss    += loss.item()
                vcorrect += (logits.argmax(1) == y).sum().item()
                vtotal   += y.size(0)
        val_loss = vloss / len(val_loader)
        val_acc  = vcorrect / vtotal
        elapsed  = time.time() - t0

        # report & prune
        trial.report(val_loss, epoch)
        if trial.should_prune():
            print(f"▸ Trial {trial.number} pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
            f"val_loss={val_loss:.4f} acc={val_acc:.4f} | "
            f"time={elapsed:.1f}s"
        )

        # step the scheduler
        scheduler.step()

        # early stopping & record best
        if val_loss < best_val_loss:
            best_val_loss     = val_loss
            epochs_no_improve = 0
            best_train_loss   = train_loss
            best_train_acc    = train_acc
            best_val_acc      = val_acc
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"★ Early stopping at epoch {epoch}")
                break

    # store best‐epoch metrics
    trial.set_user_attr("best_train_loss", best_train_loss)
    trial.set_user_attr("best_train_acc",  best_train_acc)
    trial.set_user_attr("best_val_acc",    best_val_acc)

    wandb.finish()
    gc.collect()
    return best_val_loss


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
        study_name="eeg_holdout_trial",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_holdout.db",
        load_if_exists=True
    )
    study.optimize(objective_holdout, n_trials=30)

    # print best trial & metrics
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"best_val_loss   = {best.value:.6f}")
    print(f"best_val_acc    = {best.user_attrs['best_val_acc']:.4f}")
    print(f"best_train_loss = {best.user_attrs['best_train_loss']:.4f}")
    print(f"best_train_acc  = {best.user_attrs['best_train_acc']:.4f}")
    print("best params:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")


In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold

import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Hyperparameter ranges and fixed settings ─────────────────────────
LR_MIN, LR_MAX     = 1e-4, 1e-3             # learning rate search bounds
WD_MIN, WD_MAX     = 1e-5, 1e-3             # weight decay search bounds
NUM_FILTERS        = 120                    # fixed number of convolutional filters
NUM_BLOCK_CHOICES = [2, 3]
NUM_HEAD_CHOICES   = [2, 3, 4]                 # choices for attention heads
SEGMENT_CHOICES    = [5, 10, 15]                    # choices for segments (fixed here)

# ─── Training configuration ─────────────────────────
N_FOLDS     = 5                             # number of CV folds
MAX_EPOCHS  = 100                           # maximal epochs per fold
PATIENCE    = 20                            # early stopping patience
BATCH_SIZE  = 32                            # batch size
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))  # data loader workers

# ─── Data paths & device ─────────────────────────
DATA_DIR   = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE = "labels.json"
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def objective(trial):
    """
    Optuna objective function: samples hyperparams, runs K-fold CV,
    reports validation loss for pruning, and returns average val_loss.
    """
    # 1) Sample hyperparameters from defined ranges
    lr           = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_filters  = NUM_FILTERS
    num_blocks   = trial.suggest_categorical("num_blocks", NUM_BLOCK_CHOICES)
    num_heads    = trial.suggest_categorical("num_heads", NUM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    print(f"\n===== Trial {trial.number} start =====")
    print(f"  lr={lr:.2e}, weight_decay={weight_decay:.2e}, "
          f"num_blocks={num_blocks}, num_heads={num_heads}, num_segments={num_segments}")

    # 2) Load metadata and build the full dataset to infer input_length
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds    = EEGDataset(DATA_DIR, train_meta)
    labels     = [d["label"] for d in train_meta]
    n_samples  = len(full_ds)

    # Determine original sequence length before any convolution
    input_length = full_ds[0][0].shape[-1]

    # 3) Initialize Weights & Biases for logging (one run per trial)
    wandb.init(project="eeg-cv-tuning-trial-3", config=trial.params)

    # 4) Set up Stratified K-Fold cross-validation
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    # Dictionary to collect per-fold metrics
    fold_metrics = {k: [] for k in ["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"]}

    # Loop over each fold
    for fold, (train_idx, val_idx) in enumerate(skf.split(range(n_samples), labels)):
        print(f"\n--- Fold {fold} ---")

        # Create DataLoaders for this fold
        train_loader = DataLoader(
            Subset(full_ds, train_idx),
            batch_size=BATCH_SIZE,
            shuffle=True,
            num_workers=NUM_WORKERS
        )
        val_loader = DataLoader(
            Subset(full_ds, val_idx),
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=NUM_WORKERS
        )

        # 4a) Instantiate model, optimizer, and loss
        model = EEGformer(
            in_channels  = 19,
            input_length = input_length,
            kernel_size  = 10,
            num_filters  = num_filters,
            num_heads    = num_heads,
            num_blocks   = num_blocks,
            num_segments = num_segments,
            num_classes  = 3
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        # Variables for early stopping
        best_val_loss     = float("inf")
        epochs_no_improve = 0
        best_epoch        = 0
        best_train_l = best_train_a = best_val_a = None

        # 4b) Training loop per epoch
        for epoch in range(1, MAX_EPOCHS + 1):
            t0 = time.time()

            # — Training phase —
            model.train()
            train_loss_sum = train_correct = train_total = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss   = criterion(logits, y)
                loss.backward()
                optimizer.step()

                train_loss_sum += loss.item()
                train_correct  += (logits.argmax(1) == y).sum().item()
                train_total    += y.size(0)

            train_loss = train_loss_sum / len(train_loader)
            train_acc  = train_correct / train_total

            # — Validation phase —
            model.eval()
            val_loss_sum = val_correct = val_total = 0
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss   = criterion(logits, y)
                    val_loss_sum += loss.item()
                    val_correct  += (logits.argmax(1) == y).sum().item()
                    val_total    += y.size(0)

            val_loss = val_loss_sum / len(val_loader)
            val_acc  = val_correct / val_total
            epoch_time = time.time() - t0

            # Report intermediate objective value to Optuna for pruning
            step = fold * MAX_EPOCHS + epoch
            trial.report(val_loss, step=step)
            if trial.should_prune():
                print(f"▸ Trial pruned at fold {fold}, epoch {epoch}")
                raise optuna.TrialPruned()

            # Log metrics and epoch time
            print(
                f"[Fold {fold}] Epoch {epoch:03d} "
                f"train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
                f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} | "
                f"time={epoch_time:.1f}s"
            )

            # Early stopping check
            if val_loss < best_val_loss:
                best_val_loss     = val_loss
                best_epoch        = epoch
                epochs_no_improve = 0
                best_train_l      = train_loss
                best_train_a      = train_acc
                best_val_a        = val_acc
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"[Fold {fold}] early stopping at epoch {epoch}")
                    raise optuna.TrialPruned()

        # Record this fold's best metrics
        for key, value in zip(
            ["train_loss","train_acc","val_loss","val_acc","best_epoch"],
            [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]
        ):
            fold_metrics[key].append(value)
            trial.set_user_attr(f"fold{fold}_{key}", value)

        # Free GPU memory before next fold
        del model, optimizer, train_loader, val_loader
        torch.cuda.empty_cache()
        gc.collect()

    # 5) Compute average metrics across folds and finish the trial
    avg = lambda key: sum(fold_metrics[key]) / N_FOLDS
    for key in fold_metrics:
        trial.set_user_attr(f"avg_{key}", avg(key))

    wandb.finish()
    return avg("val_loss")


if __name__ == "__main__":
    # Allow safe multiprocessing on Windows
    multiprocessing.freeze_support()

    # Create or load an Optuna study
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
        study_name="eegformer_optuna_cv_trial_3",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eeg_optuna_trial_3.db",
        load_if_exists=True
    )
    # Run hyperparameter optimization
    study.optimize(objective, n_trials=10)

    # Print out best trial results
    best = study.best_trial
    print("\n===== Best Trial =====")
    print(f"avg_val_loss   = {best.value:.6f}")
    print(f"avg_train_acc  = {best.user_attrs['avg_train_acc']:.4f}")
    print(f"avg_val_acc    = {best.user_attrs['avg_val_acc']:.4f}")
    print("best hyperparameters:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")

    # ─── Retrain on full training data using best hyperparameters ─────────────────────────
    print("\nRetraining on full TRAIN set with best hyperparams…")
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    full_meta   = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, full_meta)
    full_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

    # Determine full input length
    input_length = full_ds[0][0].shape[-1]

    # Re-instantiate model with best params
    model = EEGformer(
        in_channels  = 19,
        input_length = input_length,
        kernel_size  = 10,
        num_filters  = NUM_FILTERS,
        num_heads    = best.params["num_heads"],
        num_blocks   = best.params["num_blocks"],
        num_segments = best.params["num_segments"],
        num_classes  = 3
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best.params["lr"],
        weight_decay=best.params["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # Final full-dataset training loop
    for epoch in range(1, MAX_EPOCHS + 1):
        t0 = time.time()
        model.train()
        loss_sum = correct = total = 0
        for X, y in full_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            correct  += (logits.argmax(1) == y).sum().item()
            total    += y.size(0)

        avg_loss = loss_sum / len(full_loader)
        acc      = correct / total
        epoch_time = time.time() - t0
        print(f"[Full Train] Epoch {epoch:03d} | loss={avg_loss:.4f} | acc={acc:.4f} | time={epoch_time:.1f}s")

    # Save the final checkpoint
    ckpt_dir = '/content/drive/MyDrive/2025_Lab_Research/checkpoints'
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, 'eegformer_best_3.pth')
    torch.save(model.state_dict(), ckpt_path)
    print(f"💾 Saved best model to {ckpt_path}")


[I 2025-04-30 09:49:03,595] A new study created in RDB with name: eegformer_optuna_cv_trial_3



===== Trial 0 start =====
  lr=3.15e-05, weight_decay=1.45e-06, num_blocks=3, num_heads=3, num_segments=25



--- Fold 0 ---
[Fold 0] Epoch 001 train_loss=1.0734 train_acc=0.4311 | val_loss=1.0814 val_acc=0.4317 | time=39.0s
[Fold 0] Epoch 002 train_loss=1.0672 train_acc=0.4311 | val_loss=1.0830 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 003 train_loss=1.0674 train_acc=0.4311 | val_loss=1.0846 val_acc=0.4317 | time=38.9s
[Fold 0] Epoch 004 train_loss=1.0662 train_acc=0.4311 | val_loss=1.0832 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 005 train_loss=1.0661 train_acc=0.4311 | val_loss=1.0829 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 006 train_loss=1.0667 train_acc=0.4311 | val_loss=1.0842 val_acc=0.4317 | time=38.9s
[Fold 0] Epoch 007 train_loss=1.0666 train_acc=0.4311 | val_loss=1.0855 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 008 train_loss=1.0665 train_acc=0.4311 | val_loss=1.0838 val_acc=0.4317 | time=38.7s
[Fold 0] Epoch 009 train_loss=1.0668 train_acc=0.4311 | val_loss=1.0835 val_acc=0.4317 | time=38.7s
[Fold 0] Epoch 010 train_loss=1.0666 train_acc=0.4311 | val_loss=1.0829 val_acc=0.43

[I 2025-04-30 10:02:44,729] Trial 0 pruned. 


[Fold 0] Epoch 021 train_loss=1.0671 train_acc=0.4311 | val_loss=1.0846 val_acc=0.4317 | time=38.9s
[Fold 0] early stopping at epoch 21

===== Trial 1 start =====
  lr=1.40e-05, weight_decay=5.99e-05, num_blocks=3, num_heads=3, num_segments=25


...

[Fold 0] Epoch 021 train_loss=1.0659 train_acc=0.4311 | val_loss=1.0839 val_acc=0.4317 | time=38.9s
[Fold 0] early stopping at epoch 21

===== Trial 9 start =====
  lr=2.54e-06, weight_decay=1.58e-05, num_blocks=3, num_heads=3, num_segments=25



--- Fold 0 ---
[Fold 0] Epoch 001 train_loss=1.0822 train_acc=0.4311 | val_loss=1.0845 val_acc=0.4317 | time=38.9s
[Fold 0] Epoch 002 train_loss=1.0763 train_acc=0.4311 | val_loss=1.0824 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 003 train_loss=1.0735 train_acc=0.4311 | val_loss=1.0813 val_acc=0.4317 | time=38.8s
[Fold 0] Epoch 004 train_loss=1.0708 train_acc=0.4311 | val_loss=1.0809 val_acc=0.4317 | time=39.0s
[Fold 0] Epoch 005 train_loss=1.0691 train_acc=0.4311 | val_loss=1.0808 val_acc=0.4317 | time=39.0s
[Fold 0] Epoch 006 train_loss=1.0681 train_acc=0.4311 | val_loss=1.0809 val_acc=0.4317 | time=39.0s
[Fold 0] Epoch 007 train_loss=1.0680 train_acc=0.4311 | val_loss=1.0811 val_acc=0.4317 | time=39.1s
[Fold 0] Epoch 008 train_loss=1.0673 train_acc=0.4311 | val_loss=1.0813 val_acc=0.4317 | time=38.7s
[Fold 0] Epoch 009 train_loss=1.0668 train_acc=0.4311 | val_loss=1.0815 val_acc=0.4317 | time=38.6s
[Fold 0] Epoch 010 train_loss=1.0672 train_acc=0.4311 | val_loss=1.0817 val_acc=0.43

[I 2025-04-30 13:36:46,965] Trial 9 pruned. 


[Fold 0] Epoch 025 train_loss=1.0668 train_acc=0.4311 | val_loss=1.0835 val_acc=0.4317 | time=39.0s
[Fold 0] early stopping at epoch 25


ValueError: Record does not exist.

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold

import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Constants ─────────────────────────
LR_MIN, LR_MAX = 2e-5, 8e-5
WD_MIN, WD_MAX = 3e-5, 1e-4
FILTER_MIN, FILTER_MAX, FILTER_STEP = 60, 180, 60
HEAD_CHOICES    = [2, 4]
SEGMENT_CHOICES = [5, 15, 25]

N_FOLDS     = 5
MAX_EPOCHS  = 100
PATIENCE    = 30
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = "labels.json"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Hyperparameter search space setup
    lr = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_filters = trial.suggest_int("num_filters", FILTER_MIN, FILTER_MAX, step=FILTER_STEP)
    rtm_blocks = trial.suggest_int("rtm_blocks", 1, 3)
    stm_blocks = trial.suggest_int("stm_blocks", 1, 3)
    ttm_blocks = trial.suggest_int("ttm_blocks", 1, 3)
    rtm_heads = trial.suggest_categorical("rtm_heads", HEAD_CHOICES)
    stm_heads = trial.suggest_categorical("stm_heads", HEAD_CHOICES)
    ttm_heads = trial.suggest_categorical("ttm_heads", HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # Pruning condition: only proceed if num_filters is divisible by heads
    for h in (rtm_heads, stm_heads, ttm_heads):
        if num_filters % h != 0:
            raise optuna.TrialPruned()

    # Initialize wandb
    wandb.init(project="eeg-cv-tuning-trial_9", config=trial.params)

    print(f"\n========================= Trial {trial.number} =========================")
    print(f"Testing with hyperparameters: {trial.params}")

    # Data preparation
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds = EEGDataset(DATA_DIR, train_meta)
    labels = [d["label"] for d in train_meta]
    n_samples = len(full_ds)

    # StratifiedKFold setup
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    fold_metrics = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "best_epoch": []}

    for fold, (train_idx, val_idx) in enumerate(skf.split(range(n_samples), labels)):
        # Fold separation
        print(f"\n========================= Fold {fold} =========================")

        # Data loader setup
        train_loader = DataLoader(Subset(full_ds, train_idx), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(Subset(full_ds, val_idx), batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Model and optimizer setup
        model = EEGformer(
            num_classes=3,
            in_channels=19,
            kernel_size=10,
            num_filters=num_filters,
            rtm_blocks=rtm_blocks,
            stm_blocks=stm_blocks,
            ttm_blocks=ttm_blocks,
            rtm_heads=rtm_heads,
            stm_heads=stm_heads,
            ttm_heads=ttm_heads,
            num_segments=num_segments
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        best_val_loss = float("inf")
        epochs_no_improve = 0
        best_epoch = 0
        best_train_l = best_train_a = best_val_a = None
        last_log_time = time.time()

        # Epoch-wise training
        for epoch in range(1, MAX_EPOCHS + 1):
            model.train()
            tl_sum = t_corr = t_tot = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                tl_sum += loss.item()
                t_corr += (logits.argmax(1) == y).sum().item()
                t_tot += y.size(0)

            train_loss = tl_sum / len(train_loader)
            train_acc = t_corr / t_tot

            model.eval()
            vl_sum = v_corr = v_tot = 0
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss = criterion(logits, y)
                    vl_sum += loss.item()
                    v_corr += (logits.argmax(1) == y).sum().item()
                    v_tot += y.size(0)

            val_loss = vl_sum / len(val_loader)
            val_acc = v_corr / v_tot

            step = fold * MAX_EPOCHS + epoch
            trial.report(val_loss, step=step)

            # Pruning check
            if trial.should_prune():
                print(f"\u274c Trial {trial.number} pruned at fold {fold}, epoch {epoch}")
                # Report the metrics before returning early
                for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                 [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                    fold_metrics[k].append(v)
                    trial.set_user_attr(f"fold{fold}_{k}", v)
                raise optuna.TrialPruned()  # End trial completely if pruned

            now = time.time()
            print(f"[Fold {fold}] Epoch {epoch:03d} | train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
                  f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={now - last_log_time:.1f}s")
            last_log_time = now

            # Early stopping: if validation loss does not improve
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
                epochs_no_improve = 0
                best_train_l = train_loss
                best_train_a = train_acc
                best_val_a = val_acc
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"[Fold {fold}] Early stopping at epoch {epoch}, best was {best_epoch}")
                    # Report the metrics before returning early
                    for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                     [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                        fold_metrics[k].append(v)
                        trial.set_user_attr(f"fold{fold}_{k}", v)
                    return best_val_loss  # End trial completely if early stopping

        # Record results for the fold
        for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                         [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
            fold_metrics[k].append(v)
            trial.set_user_attr(f"fold{fold}_{k}", v)

        del model, optimizer, train_loader, val_loader
        torch.mps.empty_cache() if DEVICE.type == "mps" else torch.cuda.empty_cache()
        gc.collect()

    # Calculate average metrics
    avg = lambda k: sum(fold_metrics[k]) / N_FOLDS
    for key in ["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"]:
        trial.set_user_attr(f"avg_{key}", avg(key))

    wandb.finish()
    return avg("val_loss")


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
        study_name="eegformer_optuna_cv_3",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eegformer_optuna_cv_3.db",
        load_if_exists=True
    )
    study.optimize(objective, n_trials=8)

    best = study.best_trial
    print("\n===== Best Trial Results =====")
    print(f"avg_val_loss   = {best.value:.6f}")
    print(f"avg_train_loss = {best.user_attrs['avg_train_loss']:.6f}")
    print(f"avg_train_acc  = {best.user_attrs['avg_train_acc']:.4f}")
    print(f"avg_val_acc    = {best.user_attrs['avg_val_acc']:.4f}")
    print(f"avg_best_epoch = {best.user_attrs['avg_best_epoch']:.1f}")
    print("best hyperparameters:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")
    print("per-fold best metrics:")
    for f in range(N_FOLDS):
        print(
            f"  Fold {f}: epoch={best.user_attrs[f'fold{f}_best_epoch']}, "
            f"t_loss={best.user_attrs[f'fold{f}_train_loss']:.4f}, "
            f"t_acc={best.user_attrs[f'fold{f}_train_acc']:.4f}, "
            f"v_loss={best.user_attrs[f'fold{f}_val_loss']:.4f}, "
            f"v_acc={best.user_attrs[f'fold{f}_val_acc']:.4f}"
        )


[I 2025-04-28 01:54:11,503] A new study created in RDB with name: eegformer_optuna_cv_3



========================= Trial 0 =========================
Testing with hyperparameters: {'lr': 6.273724067568952e-05, 'weight_decay': 5.8339962733943407e-05, 'num_filters': 120, 'rtm_blocks': 1, 'stm_blocks': 3, 'ttm_blocks': 1, 'rtm_heads': 4, 'stm_heads': 2, 'ttm_heads': 2, 'num_segments': 5}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0687 train_acc=0.4318 | val_loss=1.0820 val_acc=0.4317 | time=1308.5s
[Fold 0] Epoch 002 | train_loss=1.0534 train_acc=0.4590 | val_loss=1.0284 val_acc=0.5435 | time=61.5s
[Fold 0] Epoch 003 | train_loss=1.0157 train_acc=0.5153 | val_loss=1.0012 val_acc=0.5481 | time=64.1s
[Fold 0] Epoch 004 | train_loss=0.9863 train_acc=0.5480 | val_loss=0.9923 val_acc=0.5528 | time=62.7s
[Fold 0] Epoch 005 | train_loss=0.9767 train_acc=0.5515 | val_loss=0.9871 val_acc=0.5652 | time=60.3s
[Fold 0] Epoch 006 | train_loss=0.9585 train_acc=0.5666 | val_loss=0.9754 val_acc=0.5652 | time=60.8s
[Fold 0] Epoch 007 | train_

[W 2025-04-28 03:07:14,457] Trial 0 failed with parameters: {'lr': 6.273724067568952e-05, 'weight_decay': 5.8339962733943407e-05, 'num_filters': 120, 'rtm_blocks': 1, 'stm_blocks': 3, 'ttm_blocks': 1, 'rtm_heads': 4, 'stm_heads': 2, 'ttm_heads': 2, 'num_segments': 5} because of the following error: The value None could not be cast to float..
[W 2025-04-28 03:07:14,458] Trial 0 failed with value None.


[Fold 0] Epoch 051 | train_loss=0.2137 train_acc=0.9227 | val_loss=0.5547 val_acc=0.8199 | time=61.4s
[Fold 0] Early stopping at epoch 51, best was 36



========================= Trial 1 =========================
Testing with hyperparameters: {'lr': 2.1557163203444662e-05, 'weight_decay': 0.000159517315156258, 'num_filters': 120, 'rtm_blocks': 1, 'stm_blocks': 1, 'ttm_blocks': 3, 'rtm_heads': 2, 'stm_heads': 4, 'ttm_heads': 4, 'num_segments': 25}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0685 train_acc=0.4311 | val_loss=1.0841 val_acc=0.4317 | time=59.4s
[Fold 0] Epoch 002 | train_loss=1.0668 train_acc=0.4311 | val_loss=1.0834 val_acc=0.4317 | time=60.0s
[Fold 0] Epoch 003 | train_loss=1.0662 train_acc=0.4311 | val_loss=1.0836 val_acc=0.4317 | time=62.5s
[Fold 0] Epoch 004 | train_loss=1.0651 train_acc=0.4311 | val_loss=1.0864 val_acc=0.4317 | time=61.6s
[Fold 0] Epoch 005 | train_loss=1.0668 train_acc=0.4311 | val_loss=1.0853 val_acc=0.4317 | time=59.7s
[Fold 0] Epoch 006 | train_loss=1.0657 train_acc=0.4311 | val_loss=1.0818 val_acc=0.4317 | time=60.3s
[Fold 0] Epoch 007 | train_lo

[W 2025-04-28 04:39:30,295] Trial 1 failed with parameters: {'lr': 2.1557163203444662e-05, 'weight_decay': 0.000159517315156258, 'num_filters': 120, 'rtm_blocks': 1, 'stm_blocks': 1, 'ttm_blocks': 3, 'rtm_heads': 2, 'stm_heads': 4, 'ttm_heads': 4, 'num_segments': 25} because of the following error: The value None could not be cast to float..
[W 2025-04-28 04:39:30,297] Trial 1 failed with value None.


[Fold 0] Epoch 094 | train_loss=0.1312 train_acc=0.9546 | val_loss=0.6095 val_acc=0.8199 | time=56.7s
[Fold 0] Early stopping at epoch 94, best was 79



========================= Trial 2 =========================
Testing with hyperparameters: {'lr': 1.2400547483173527e-05, 'weight_decay': 7.79511927028319e-05, 'num_filters': 120, 'rtm_blocks': 3, 'stm_blocks': 3, 'ttm_blocks': 1, 'rtm_heads': 2, 'stm_heads': 4, 'ttm_heads': 4, 'num_segments': 15}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0852 train_acc=0.4155 | val_loss=1.0800 val_acc=0.4317 | time=72.3s
[Fold 0] Epoch 002 | train_loss=1.0672 train_acc=0.4311 | val_loss=1.0815 val_acc=0.4317 | time=72.3s
[Fold 0] Epoch 003 | train_loss=1.0667 train_acc=0.4311 | val_loss=1.0817 val_acc=0.4317 | time=72.3s
[Fold 0] Epoch 004 | train_loss=1.0649 train_acc=0.4311 | val_loss=1.0811 val_acc=0.4317 | time=71.7s
[Fold 0] Epoch 005 | train_loss=1.0639 train_acc=0.4311 | val_loss=1.0798 val_acc=0.4317 | time=72.2s
[Fold 0] Epoch 006 | train_loss=1.0617 train_acc=0.4330 | val_loss=1.0696 val_acc=0.4317 | time=71.7s
[Fold 0] Epoch 007 | train_lo

In [ ]:
from eeg_dataset import EEGDataset
from models import EEGformer

Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold

import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Constants ─────────────────────────
LR_MIN, LR_MAX = 2e-5, 8e-5
WD_MIN, WD_MAX = 3e-5, 1e-4
FILTER_MIN = FILTER_MAX = 120

# blocks -> [1,2,3] / heads -> [2,3]
RTM_BLOCK_CHOICES = [1, 2, 3]
STM_BLOCK_CHOICES = [1, 2, 3]
TTM_BLOCK_CHOICES = [1, 2, 3]

RTM_HEAD_CHOICES = [2, 3, 4]
STM_HEAD_CHOICES = [2, 3, 4]
TTM_HEAD_CHOICES = [2, 3, 4]

SEGMENT_CHOICES = [5]

N_FOLDS     = 5
MAX_EPOCHS  = 100
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = "labels.json"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Hyperparameter search space setup
    lr = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_filters = 120
    rtm_blocks = trial.suggest_categorical("rtm_blocks", RTM_BLOCK_CHOICES)
    stm_blocks = trial.suggest_categorical("stm_blocks", STM_BLOCK_CHOICES)
    ttm_blocks = trial.suggest_categorical("ttm_blocks", TTM_BLOCK_CHOICES)
    rtm_heads = trial.suggest_categorical("rtm_heads", RTM_HEAD_CHOICES)
    stm_heads = trial.suggest_categorical("stm_heads", STM_HEAD_CHOICES)
    ttm_heads = trial.suggest_categorical("ttm_heads", TTM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # Pruning condition: only proceed if num_filters is divisible by heads
    for h in (rtm_heads, stm_heads, ttm_heads):
        if num_filters % h != 0:
            raise optuna.TrialPruned()

    # Initialize wandb
    wandb.init(project="eeg-cv-tuning-trial_10", config=trial.params)

    print(f"\n========================= Trial {trial.number} =========================")
    print(f"Testing with hyperparameters: {trial.params}")

    # Data preparation
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds = EEGDataset(DATA_DIR, train_meta)
    labels = [d["label"] for d in train_meta]
    n_samples = len(full_ds)

    # StratifiedKFold setup
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    fold_metrics = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "best_epoch": []}

    for fold, (train_idx, val_idx) in enumerate(skf.split(range(n_samples), labels)):
        # Fold separation
        print(f"\n========================= Fold {fold} =========================")

        # Data loader setup
        train_loader = DataLoader(Subset(full_ds, train_idx), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(Subset(full_ds, val_idx), batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Model and optimizer setup
        model = EEGformer(
            num_classes=3,
            in_channels=19,
            kernel_size=10,
            num_filters=num_filters,
            rtm_blocks=rtm_blocks,
            stm_blocks=stm_blocks,
            ttm_blocks=ttm_blocks,
            rtm_heads=rtm_heads,
            stm_heads=stm_heads,
            ttm_heads=ttm_heads,
            num_segments=num_segments
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        best_val_loss = float("inf")
        epochs_no_improve = 0
        best_epoch = 0
        best_train_l = best_train_a = best_val_a = None
        last_log_time = time.time()

        # Epoch-wise training
        for epoch in range(1, MAX_EPOCHS + 1):
            model.train()
            tl_sum = t_corr = t_tot = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                tl_sum += loss.item()
                t_corr += (logits.argmax(1) == y).sum().item()
                t_tot += y.size(0)

            train_loss = tl_sum / len(train_loader)
            train_acc = t_corr / t_tot

            model.eval()
            vl_sum = v_corr = v_tot = 0
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss = criterion(logits, y)
                    vl_sum += loss.item()
                    v_corr += (logits.argmax(1) == y).sum().item()
                    v_tot += y.size(0)

            val_loss = vl_sum / len(val_loader)
            val_acc = v_corr / v_tot

            step = fold * MAX_EPOCHS + epoch
            trial.report(val_loss, step=step)

            # Pruning check
            if trial.should_prune():
                print(f"\u274c Trial {trial.number} pruned at fold {fold}, epoch {epoch}")
                # Report the metrics before returning early
                for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                 [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                    fold_metrics[k].append(v)
                    trial.set_user_attr(f"fold{fold}_{k}", v)
                raise optuna.TrialPruned()  # End trial completely if pruned

            now = time.time()
            print(f"[Fold {fold}] Epoch {epoch:03d} | train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
                  f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={now - last_log_time:.1f}s")
            last_log_time = now

            # Early stopping: if validation loss does not improve
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
                epochs_no_improve = 0
                best_train_l = train_loss
                best_train_a = train_acc
                best_val_a = val_acc
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"[Fold {fold}] Early stopping at epoch {epoch}, best was {best_epoch}")
                    # Report the metrics before returning early
                    for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                     [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                        fold_metrics[k].append(v)
                        trial.set_user_attr(f"fold{fold}_{k}", v)
                    return best_val_loss  # End trial completely if early stopping

        # Record results for the fold
        for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                         [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
            fold_metrics[k].append(v)
            trial.set_user_attr(f"fold{fold}_{k}", v)

        del model, optimizer, train_loader, val_loader
        torch.mps.empty_cache() if DEVICE.type == "mps" else torch.cuda.empty_cache()
        gc.collect()

    # Calculate average metrics
    avg = lambda k: sum(fold_metrics[k]) / N_FOLDS
    for key in ["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"]:
        trial.set_user_attr(f"avg_{key}", avg(key))

    wandb.finish()
    return avg("val_loss")


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
        study_name="eegformer_optuna_cv_4",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eegformer_optuna_cv_4.db",
        load_if_exists=True
    )
    study.optimize(objective, n_trials=10)

    best = study.best_trial
    print("\n===== Best Trial Results =====")
    print(f"avg_val_loss   = {best.value:.6f}")
    print(f"avg_train_loss = {best.user_attrs['avg_train_loss']:.6f}")
    print(f"avg_train_acc  = {best.user_attrs['avg_train_acc']:.4f}")
    print(f"avg_val_acc    = {best.user_attrs['avg_val_acc']:.4f}")
    print(f"avg_best_epoch = {best.user_attrs['avg_best_epoch']:.1f}")
    print("best hyperparameters:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")
    print("per-fold best metrics:")
    for f in range(N_FOLDS):
        print(
            f"  Fold {f}: epoch={best.user_attrs[f'fold{f}_best_epoch']}, "
            f"t_loss={best.user_attrs[f'fold{f}_train_loss']:.4f}, "
            f"t_acc={best.user_attrs[f'fold{f}_train_acc']:.4f}, "
            f"v_loss={best.user_attrs[f'fold{f}_val_loss']:.4f}, "
            f"v_acc={best.user_attrs[f'fold{f}_val_acc']:.4f}"
        )

    # ─── 전체 데이터(train)로 재학습 + loss·accuracy 출력 + 저장 ─────────────────
    print("\nRetraining on full TRAIN dataset with best params…")

    # 1) train 메타만 골라서 로드
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    full_meta   = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, full_meta)
    full_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

    # 2) 모델·옵티마이저 재설정
    model = EEGformer(
        num_classes=3,
        in_channels=19,
        kernel_size=10,
        num_filters=120,
        rtm_blocks=best.params["rtm_blocks"],
        stm_blocks=best.params["stm_blocks"],
        ttm_blocks=best.params["ttm_blocks"],
        rtm_heads= best.params["rtm_heads"],
        stm_heads= best.params["stm_heads"],
        ttm_heads= best.params["ttm_heads"],
        num_segments=best.params["num_segments"]
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best.params["lr"],
        weight_decay=best.params["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # 3) MAX_EPOCHS 만큼 전체 학습하며 loss·accuracy 출력
    for epoch in range(1, MAX_EPOCHS + 1):
        model.train()
        loss_sum = 0.0
        correct  = 0
        total    = 0
        for X, y in full_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            correct  += (logits.argmax(1) == y).sum().item()
            total    += y.size(0)

        avg_loss = loss_sum / len(full_loader)
        acc      = correct / total
        print(f"[Full Train] Epoch {epoch:03d} | loss={avg_loss:.4f} | acc={acc:.4f}")

    # 4) 체크포인트 저장
    ckpt_dir = '/content/drive/MyDrive/2025_Lab_Research/checkpoints'
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, 'eegformer_best.pth')
    torch.save(model.state_dict(), ckpt_path)
    print(f"💾 Saved best model to {ckpt_path}")


[I 2025-04-28 07:20:44,501] A new study created in RDB with name: eegformer_optuna_cv_4



========================= Trial 0 =========================
Testing with hyperparameters: {'lr': 7.00045782400034e-05, 'weight_decay': 8.734906501291418e-05, 'rtm_blocks': 1, 'stm_blocks': 2, 'ttm_blocks': 2, 'rtm_heads': 2, 'stm_heads': 3, 'ttm_heads': 2, 'num_segments': 5}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0691 train_acc=0.4225 | val_loss=1.0870 val_acc=0.4317 | time=726.1s
[Fold 0] Epoch 002 | train_loss=1.0673 train_acc=0.4311 | val_loss=1.0835 val_acc=0.4317 | time=18.7s
[Fold 0] Epoch 003 | train_loss=1.0675 train_acc=0.4311 | val_loss=1.0879 val_acc=0.4317 | time=18.4s
[Fold 0] Epoch 004 | train_loss=1.0671 train_acc=0.4311 | val_loss=1.0844 val_acc=0.4317 | time=18.6s
[Fold 0] Epoch 005 | train_loss=1.0672 train_acc=0.4311 | val_loss=1.0842 val_acc=0.4317 | time=18.2s
[Fold 0] Epoch 006 | train_loss=1.0672 train_acc=0.4311 | val_loss=1.0854 val_acc=0.4317 | time=18.2s
[Fold 0] Epoch 007 | train_loss=1.0665 train_acc=0

[I 2025-04-28 07:52:34,910] Trial 0 finished with value: 0.7909612684022813 and parameters: {'lr': 7.00045782400034e-05, 'weight_decay': 8.734906501291418e-05, 'rtm_blocks': 1, 'stm_blocks': 2, 'ttm_blocks': 2, 'rtm_heads': 2, 'stm_heads': 3, 'ttm_heads': 2, 'num_segments': 5}. Best is trial 0 with value: 0.7909612684022813.


[Fold 0] Epoch 065 | train_loss=0.1856 train_acc=0.9262 | val_loss=1.1634 val_acc=0.7189 | time=17.5s
[Fold 0] Early stopping at epoch 65, best was 45



========================= Trial 1 =========================
Testing with hyperparameters: {'lr': 4.9554878139823925e-05, 'weight_decay': 8.001690176093237e-05, 'rtm_blocks': 1, 'stm_blocks': 3, 'ttm_blocks': 2, 'rtm_heads': 2, 'stm_heads': 2, 'ttm_heads': 2, 'num_segments': 5}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0684 train_acc=0.4311 | val_loss=1.0891 val_acc=0.4317 | time=17.9s
[Fold 0] Epoch 002 | train_loss=1.0669 train_acc=0.4311 | val_loss=1.0846 val_acc=0.4317 | time=17.7s
[Fold 0] Epoch 003 | train_loss=1.0657 train_acc=0.4311 | val_loss=1.0803 val_acc=0.4317 | time=17.6s
[Fold 0] Epoch 004 | train_loss=1.0626 train_acc=0.4338 | val_loss=1.0817 val_acc=0.4317 | time=17.9s
[Fold 0] Epoch 005 | train_loss=1.0621 train_acc=0.4361 | val_loss=1.0733 val_acc=0.4410 | time=17.8s
[Fold 0] Epoch 006 | train_loss=1.0556 train_acc=0.4408 | val_loss=1.0692 val_acc=0.4394 | time=17.7s
[Fold 0] Epoch 007 | train_loss=1.0537 train_acc=

...

❌ Trial 8 pruned at fold 0, epoch 6



========================= Trial 9 =========================
Testing with hyperparameters: {'lr': 5.956687649545858e-05, 'weight_decay': 3.544220567159663e-05, 'rtm_blocks': 2, 'stm_blocks': 1, 'ttm_blocks': 3, 'rtm_heads': 2, 'stm_heads': 2, 'ttm_heads': 3, 'num_segments': 5}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0691 train_acc=0.4311 | val_loss=1.0825 val_acc=0.4317 | time=17.7s
[Fold 0] Epoch 002 | train_loss=1.0667 train_acc=0.4311 | val_loss=1.0808 val_acc=0.4317 | time=17.7s
[Fold 0] Epoch 003 | train_loss=1.0668 train_acc=0.4311 | val_loss=1.0794 val_acc=0.4317 | time=17.6s
[Fold 0] Epoch 004 | train_loss=1.0630 train_acc=0.4439 | val_loss=1.0588 val_acc=0.5326 | time=17.7s
[Fold 0] Epoch 005 | train_loss=1.0626 train_acc=0.4276 | val_loss=1.0787 val_acc=0.4317 | time=17.5s
[Fold 0] Epoch 006 | train_loss=1.0507 train_acc=0.4567 | val_loss=1.1327 val_acc=0.3432 | time=17.5s
[Fold 0] Epoch 007 | train_loss=1.0730 train_acc=0

[I 2025-04-28 10:36:27,042] Trial 9 pruned. 


❌ Trial 9 pruned at fold 0, epoch 8

===== Best Trial Results =====
avg_val_loss   = 0.614353


KeyError: 'avg_train_loss'

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold

import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Constants ─────────────────────────
LR_MIN, LR_MAX = 2e-5, 8e-5
WD_MIN, WD_MAX = 3e-5, 1e-4
FILTER_MIN = FILTER_MAX = 120

# blocks -> [1,2,3] / heads -> [2,3,4]
RTM_BLOCK_CHOICES = [1, 2, 3]
STM_BLOCK_CHOICES = [1, 2, 3]
TTM_BLOCK_CHOICES = [1, 2, 3]

RTM_HEAD_CHOICES = [2, 3, 4]
STM_HEAD_CHOICES = [2, 3, 4]
TTM_HEAD_CHOICES = [2, 3, 4]

SEGMENT_CHOICES = [5, 15]

N_FOLDS     = 5
MAX_EPOCHS  = 100
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = "labels.json"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Hyperparameter search space setup
    lr = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_filters = 120
    rtm_blocks = trial.suggest_categorical("rtm_blocks", RTM_BLOCK_CHOICES)
    stm_blocks = trial.suggest_categorical("stm_blocks", STM_BLOCK_CHOICES)
    ttm_blocks = trial.suggest_categorical("ttm_blocks", TTM_BLOCK_CHOICES)
    rtm_heads = trial.suggest_categorical("rtm_heads", RTM_HEAD_CHOICES)
    stm_heads = trial.suggest_categorical("stm_heads", STM_HEAD_CHOICES)
    ttm_heads = trial.suggest_categorical("ttm_heads", TTM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # Pruning condition: only proceed if num_filters is divisible by heads
    for h in (rtm_heads, stm_heads, ttm_heads):
        if num_filters % h != 0:
            raise optuna.TrialPruned()

    # Initialize wandb
    wandb.init(project="eeg-cv-tuning-trial_12", config=trial.params)

    print(f"\n========================= Trial {trial.number} =========================")
    print(f"Testing with hyperparameters: {trial.params}")

    # Data preparation
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds = EEGDataset(DATA_DIR, train_meta)
    labels = [d["label"] for d in train_meta]
    n_samples = len(full_ds)

    # StratifiedKFold setup
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    fold_metrics = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "best_epoch": []}

    for fold, (train_idx, val_idx) in enumerate(skf.split(range(n_samples), labels)):
        # Fold separation
        print(f"\n========================= Fold {fold} =========================")

        # Data loader setup
        train_loader = DataLoader(Subset(full_ds, train_idx), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(Subset(full_ds, val_idx), batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Model and optimizer setup
        model = EEGformer(
            num_classes=3,
            in_channels=19,
            kernel_size=10,
            num_filters=num_filters,
            rtm_blocks=rtm_blocks,
            stm_blocks=stm_blocks,
            ttm_blocks=ttm_blocks,
            rtm_heads=rtm_heads,
            stm_heads=stm_heads,
            ttm_heads=ttm_heads,
            num_segments=num_segments
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        best_val_loss = float("inf")
        epochs_no_improve = 0
        best_epoch = 0
        best_train_l = best_train_a = best_val_a = None
        last_log_time = time.time()

        # Epoch-wise training
        for epoch in range(1, MAX_EPOCHS + 1):
            model.train()
            tl_sum = t_corr = t_tot = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                tl_sum += loss.item()
                t_corr += (logits.argmax(1) == y).sum().item()
                t_tot += y.size(0)

            train_loss = tl_sum / len(train_loader)
            train_acc = t_corr / t_tot

            model.eval()
            vl_sum = v_corr = v_tot = 0
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss = criterion(logits, y)
                    vl_sum += loss.item()
                    v_corr += (logits.argmax(1) == y).sum().item()
                    v_tot += y.size(0)

            val_loss = vl_sum / len(val_loader)
            val_acc = v_corr / v_tot

            step = fold * MAX_EPOCHS + epoch
            trial.report(val_loss, step=step)

            # Pruning check
            if trial.should_prune():
                print(f"\u274c Trial {trial.number} pruned at fold {fold}, epoch {epoch}")
                # Report the metrics before returning early
                for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                 [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                    fold_metrics[k].append(v)
                    trial.set_user_attr(f"fold{fold}_{k}", v)
                raise optuna.TrialPruned()  # End trial completely if pruned

            now = time.time()
            print(f"[Fold {fold}] Epoch {epoch:03d} | train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
                  f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={now - last_log_time:.1f}s")
            last_log_time = now

            # Early stopping: if validation loss does not improve
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
                epochs_no_improve = 0
                best_train_l = train_loss
                best_train_a = train_acc
                best_val_a = val_acc
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"[Fold {fold}] Early stopping at epoch {epoch}, best was {best_epoch}")
                    # Report the metrics before returning early
                    for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                     [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                        fold_metrics[k].append(v)
                        trial.set_user_attr(f"fold{fold}_{k}", v)
                    break  # Break if early stopping

        # Record results for the fold
        for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                         [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
            fold_metrics[k].append(v)
            trial.set_user_attr(f"fold{fold}_{k}", v)

        del model, optimizer, train_loader, val_loader
        torch.mps.empty_cache() if DEVICE.type == "mps" else torch.cuda.empty_cache()
        gc.collect()

    # Calculate average metrics
    avg = lambda k: sum(fold_metrics[k]) / N_FOLDS
    for key in ["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"]:
        trial.set_user_attr(f"avg_{key}", avg(key))

    wandb.finish()
    return avg("val_loss")


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
        study_name="eegformer_optuna_cv_4",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eegformer_optuna_cv_5.db",
        load_if_exists=True
    )
    study.optimize(objective, n_trials=10)

    best = study.best_trial
    print("\n===== Best Trial Results =====")
    print(f"avg_val_loss   = {best.value:.6f}")
    print(f"avg_train_loss = {best.user_attrs['avg_train_loss']:.6f}")
    print(f"avg_train_acc  = {best.user_attrs['avg_train_acc']:.4f}")
    print(f"avg_val_acc    = {best.user_attrs['avg_val_acc']:.4f}")
    print(f"avg_best_epoch = {best.user_attrs['avg_best_epoch']:.1f}")
    print("best hyperparameters:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")
    print("per-fold best metrics:")
    for f in range(N_FOLDS):
        print(
            f"  Fold {f}: epoch={best.user_attrs[f'fold{f}_best_epoch']}, "
            f"t_loss={best.user_attrs[f'fold{f}_train_loss']:.4f}, "
            f"t_acc={best.user_attrs[f'fold{f}_train_acc']:.4f}, "
            f"v_loss={best.user_attrs[f'fold{f}_val_loss']:.4f}, "
            f"v_acc={best.user_attrs[f'fold{f}_val_acc']:.4f}"
        )

    # ─── 전체 데이터(train)로 재학습 + loss·accuracy 출력 + 저장 ─────────────────
    print("\nRetraining on full TRAIN dataset with best params…")

    # 1) train 메타만 골라서 로드
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    full_meta   = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, full_meta)
    full_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

    # 2) 모델·옵티마이저 재설정
    model = EEGformer(
        num_classes=3,
        in_channels=19,
        kernel_size=10,
        num_filters=120,
        rtm_blocks=best.params["rtm_blocks"],
        stm_blocks=best.params["stm_blocks"],
        ttm_blocks=best.params["ttm_blocks"],
        rtm_heads= best.params["rtm_heads"],
        stm_heads= best.params["stm_heads"],
        ttm_heads= best.params["ttm_heads"],
        num_segments=best.params["num_segments"]
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best.params["lr"],
        weight_decay=best.params["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # 3) MAX_EPOCHS 만큼 전체 학습하며 loss·accuracy 출력
    for epoch in range(1, MAX_EPOCHS + 1):
        model.train()
        loss_sum = 0.0
        correct  = 0
        total    = 0
        for X, y in full_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            correct  += (logits.argmax(1) == y).sum().item()
            total    += y.size(0)

        avg_loss = loss_sum / len(full_loader)
        acc      = correct / total
        print(f"[Full Train] Epoch {epoch:03d} | loss={avg_loss:.4f} | acc={acc:.4f}")

    # 4) 체크포인트 저장
    ckpt_dir = '/content/drive/MyDrive/2025_Lab_Research/checkpoints'
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, 'eegformer_best.pth')
    torch.save(model.state_dict(), ckpt_path)
    print(f"💾 Saved best model to {ckpt_path}")


Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Now using CUDA device 0
Enabling CUDA with 39.14 GiB available memory


[I 2025-04-28 21:33:58,944] A new study created in RDB with name: eegformer_optuna_cv_4



========================= Trial 0 =========================
Testing with hyperparameters: {'lr': 3.306335426726131e-05, 'weight_decay': 6.37539648704434e-05, 'rtm_blocks': 3, 'stm_blocks': 2, 'ttm_blocks': 3, 'rtm_heads': 3, 'stm_heads': 3, 'ttm_heads': 2, 'num_segments': 15}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0683 train_acc=0.4311 | val_loss=1.0845 val_acc=0.4317 | time=250.0s
[Fold 0] Epoch 002 | train_loss=1.0669 train_acc=0.4311 | val_loss=1.0853 val_acc=0.4317 | time=18.0s
[Fold 0] Epoch 003 | train_loss=1.0669 train_acc=0.4311 | val_loss=1.0818 val_acc=0.4317 | time=18.1s
[Fold 0] Epoch 004 | train_loss=1.0666 train_acc=0.4311 | val_loss=1.0820 val_acc=0.4317 | time=18.1s
[Fold 0] Epoch 005 | train_loss=1.0663 train_acc=0.4311 | val_loss=1.0851 val_acc=0.4317 | time=18.2s
[Fold 0] Epoch 006 | train_loss=1.0663 train_acc=0.4311 | val_loss=1.0822 val_acc=0.4317 | time=18.1s
[Fold 0] Epoch 007 | train_loss=1.0677 train_acc=

[I 2025-04-28 23:45:36,764] Trial 0 finished with value: 1.23324632417588 and parameters: {'lr': 3.306335426726131e-05, 'weight_decay': 6.37539648704434e-05, 'rtm_blocks': 3, 'stm_blocks': 2, 'ttm_blocks': 3, 'rtm_heads': 3, 'stm_heads': 3, 'ttm_heads': 2, 'num_segments': 15}. Best is trial 0 with value: 1.23324632417588.



========================= Trial 1 =========================
Testing with hyperparameters: {'lr': 5.4741533345840924e-05, 'weight_decay': 6.0702711367702194e-05, 'rtm_blocks': 1, 'stm_blocks': 1, 'ttm_blocks': 2, 'rtm_heads': 3, 'stm_heads': 4, 'ttm_heads': 2, 'num_segments': 5}

========================= Fold 0 =========================
[Fold 0] Epoch 001 | train_loss=1.0832 train_acc=0.4171 | val_loss=1.0814 val_acc=0.4317 | time=17.3s
[Fold 0] Epoch 002 | train_loss=1.0687 train_acc=0.4276 | val_loss=1.0813 val_acc=0.4317 | time=17.2s
[Fold 0] Epoch 003 | train_loss=1.0653 train_acc=0.4361 | val_loss=1.0778 val_acc=0.4317 | time=17.5s
[Fold 0] Epoch 004 | train_loss=1.0536 train_acc=0.4571 | val_loss=1.0373 val_acc=0.5435 | time=17.5s
[Fold 0] Epoch 005 | train_loss=1.0091 train_acc=0.5297 | val_loss=1.0274 val_acc=0.5233 | time=17.6s
[Fold 0] Epoch 006 | train_loss=0.9954 train_acc=0.5371 | val_loss=0.9988 val_acc=0.5512 | time=17.8s
[Fold 0] Epoch 007 | train_loss=0.9675 train_acc

[W 2025-04-29 00:46:21,576] Trial 1 failed with parameters: {'lr': 5.4741533345840924e-05, 'weight_decay': 6.0702711367702194e-05, 'rtm_blocks': 1, 'stm_blocks': 1, 'ttm_blocks': 2, 'rtm_heads': 3, 'stm_heads': 4, 'ttm_heads': 2, 'num_segments': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-8-d413b19da3d1>", line 123, in objective
    tl_sum += loss.item()
              ^^^^^^^^^^^
KeyboardInterrupt
[W 2025-04-29 00:46:21,579] Trial 1 failed with value None.


KeyboardInterrupt: 

### Optuna 범위 줄여보기

- RTM Block = [1, 2]
- STM Block = [2, 3]
- TTM Block = [1, 2]

- RTM Head = [2, 4]
- STM Head = [2, 4]
- TTM Head = [2, 4]

- Segment Choices = [5]

In [ ]:
import os
import json
import time
import gc
import multiprocessing

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold

import optuna
import wandb

from eeg_dataset import EEGDataset
from models import EEGformer

# ─── Constants ─────────────────────────
LR_MIN, LR_MAX = 2e-5, 8e-5
WD_MIN, WD_MAX = 3e-5, 1e-4
FILTER_MIN = FILTER_MAX = 120

# blocks -> [1,2,3] / heads -> [2,3,4]
RTM_BLOCK_CHOICES = [1, 2]
STM_BLOCK_CHOICES = [2, 3]
TTM_BLOCK_CHOICES = [1, 2]

RTM_HEAD_CHOICES = [2, 4]
STM_HEAD_CHOICES = [2, 4]
TTM_HEAD_CHOICES = [2, 4]

SEGMENT_CHOICES = [5]

N_FOLDS     = 5
MAX_EPOCHS  = 100
PATIENCE    = 20
BATCH_SIZE  = 32
NUM_WORKERS = max(1, min(4, os.cpu_count() - 1))
DATA_DIR    = '/content/drive/MyDrive/2025_Lab_Research/model-data'
LABEL_FILE  = "labels.json"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Hyperparameter search space setup
    lr = trial.suggest_float("lr", LR_MIN, LR_MAX, log=True)
    weight_decay = trial.suggest_float("weight_decay", WD_MIN, WD_MAX, log=True)
    num_filters = 120
    rtm_blocks = trial.suggest_categorical("rtm_blocks", RTM_BLOCK_CHOICES)
    stm_blocks = trial.suggest_categorical("stm_blocks", STM_BLOCK_CHOICES)
    ttm_blocks = trial.suggest_categorical("ttm_blocks", TTM_BLOCK_CHOICES)
    rtm_heads = trial.suggest_categorical("rtm_heads", RTM_HEAD_CHOICES)
    stm_heads = trial.suggest_categorical("stm_heads", STM_HEAD_CHOICES)
    ttm_heads = trial.suggest_categorical("ttm_heads", TTM_HEAD_CHOICES)
    num_segments = trial.suggest_categorical("num_segments", SEGMENT_CHOICES)

    # Pruning condition: only proceed if num_filters is divisible by heads
    for h in (rtm_heads, stm_heads, ttm_heads):
        if num_filters % h != 0:
            raise optuna.TrialPruned()

    # Initialize wandb
    wandb.init(project="eeg-cv-tuning-trial_12", config=trial.params)

    print(f"\n========================= Trial {trial.number} =========================")
    print(f"Testing with hyperparameters: {trial.params}")

    # Data preparation
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    train_meta = [d for d in all_meta if d["type"] == "train"]
    full_ds = EEGDataset(DATA_DIR, train_meta)
    labels = [d["label"] for d in train_meta]
    n_samples = len(full_ds)

    # StratifiedKFold setup
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    fold_metrics = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": [], "best_epoch": []}

    for fold, (train_idx, val_idx) in enumerate(skf.split(range(n_samples), labels)):
        # Fold separation
        print(f"\n========================= Fold {fold} =========================")

        # Data loader setup
        train_loader = DataLoader(Subset(full_ds, train_idx), batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(Subset(full_ds, val_idx), batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Model and optimizer setup
        model = EEGformer(
            num_classes=3,
            in_channels=19,
            kernel_size=10,
            num_filters=num_filters,
            rtm_blocks=rtm_blocks,
            stm_blocks=stm_blocks,
            ttm_blocks=ttm_blocks,
            rtm_heads=rtm_heads,
            stm_heads=stm_heads,
            ttm_heads=ttm_heads,
            num_segments=num_segments
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.CrossEntropyLoss()

        best_val_loss = float("inf")
        epochs_no_improve = 0
        best_epoch = 0
        best_train_l = best_train_a = best_val_a = None
        last_log_time = time.time()

        # Epoch-wise training
        for epoch in range(1, MAX_EPOCHS + 1):
            model.train()
            tl_sum = t_corr = t_tot = 0
            for X, y in train_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                logits = model(X)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                tl_sum += loss.item()
                t_corr += (logits.argmax(1) == y).sum().item()
                t_tot += y.size(0)

            train_loss = tl_sum / len(train_loader)
            train_acc = t_corr / t_tot

            model.eval()
            vl_sum = v_corr = v_tot = 0
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(DEVICE), y.to(DEVICE)
                    logits = model(X)
                    loss = criterion(logits, y)
                    vl_sum += loss.item()
                    v_corr += (logits.argmax(1) == y).sum().item()
                    v_tot += y.size(0)

            val_loss = vl_sum / len(val_loader)
            val_acc = v_corr / v_tot

            step = fold * MAX_EPOCHS + epoch
            trial.report(val_loss, step=step)

            # Pruning check
            if trial.should_prune():
                print(f"\u274c Trial {trial.number} pruned at fold {fold}, epoch {epoch}")
                # Report the metrics before returning early
                for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                 [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                    fold_metrics[k].append(v)
                    trial.set_user_attr(f"fold{fold}_{k}", v)
                raise optuna.TrialPruned()  # End trial completely if pruned

            now = time.time()
            print(f"[Fold {fold}] Epoch {epoch:03d} | train_loss={train_loss:.4f} train_acc={train_acc:.4f} | "
                  f"val_loss={val_loss:.4f} val_acc={val_acc:.4f} | time={now - last_log_time:.1f}s")
            last_log_time = now

            # Early stopping: if validation loss does not improve
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
                epochs_no_improve = 0
                best_train_l = train_loss
                best_train_a = train_acc
                best_val_a = val_acc
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= PATIENCE:
                    print(f"[Fold {fold}] Early stopping at epoch {epoch}, best was {best_epoch}")
                    # Report the metrics before returning early
                    for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                                     [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
                        fold_metrics[k].append(v)
                        trial.set_user_attr(f"fold{fold}_{k}", v)
                    break  # Break if early stopping

        # Record results for the fold
        for k, v in zip(["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"],
                         [best_train_l, best_train_a, best_val_loss, best_val_a, best_epoch]):
            fold_metrics[k].append(v)
            trial.set_user_attr(f"fold{fold}_{k}", v)

        del model, optimizer, train_loader, val_loader
        torch.mps.empty_cache() if DEVICE.type == "mps" else torch.cuda.empty_cache()
        gc.collect()

    # Calculate average metrics
    avg = lambda k: sum(fold_metrics[k]) / N_FOLDS
    for key in ["train_loss", "train_acc", "val_loss", "val_acc", "best_epoch"]:
        trial.set_user_attr(f"avg_{key}", avg(key))

    wandb.finish()
    return avg("val_loss")


if __name__ == "__main__":
    multiprocessing.freeze_support()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
        study_name="eegformer_optuna_cv_4",
        storage="sqlite:////content/drive/MyDrive/2025_Lab_Research/eegformer_optuna_cv_5.db",
        load_if_exists=True
    )
    study.optimize(objective, n_trials=10)

    best = study.best_trial
    print("\n===== Best Trial Results =====")
    print(f"avg_val_loss   = {best.value:.6f}")
    print(f"avg_train_loss = {best.user_attrs['avg_train_loss']:.6f}")
    print(f"avg_train_acc  = {best.user_attrs['avg_train_acc']:.4f}")
    print(f"avg_val_acc    = {best.user_attrs['avg_val_acc']:.4f}")
    print(f"avg_best_epoch = {best.user_attrs['avg_best_epoch']:.1f}")
    print("best hyperparameters:")
    for k, v in best.params.items():
        print(f"  {k}: {v}")
    print("per-fold best metrics:")
    for f in range(N_FOLDS):
        print(
            f"  Fold {f}: epoch={best.user_attrs[f'fold{f}_best_epoch']}, "
            f"t_loss={best.user_attrs[f'fold{f}_train_loss']:.4f}, "
            f"t_acc={best.user_attrs[f'fold{f}_train_acc']:.4f}, "
            f"v_loss={best.user_attrs[f'fold{f}_val_loss']:.4f}, "
            f"v_acc={best.user_attrs[f'fold{f}_val_acc']:.4f}"
        )

    # ─── 전체 데이터(train)로 재학습 + loss·accuracy 출력 + 저장 ─────────────────
    print("\nRetraining on full TRAIN dataset with best params…")

    # 1) train 메타만 골라서 로드
    with open(os.path.join(DATA_DIR, LABEL_FILE), "r") as f:
        all_meta = json.load(f)
    full_meta   = [d for d in all_meta if d["type"] == "train"]
    full_ds     = EEGDataset(DATA_DIR, full_meta)
    full_loader = DataLoader(full_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

    # 2) 모델·옵티마이저 재설정
    model = EEGformer(
        num_classes=3,
        in_channels=19,
        kernel_size=10,
        num_filters=120,
        rtm_blocks=best.params["rtm_blocks"],
        stm_blocks=best.params["stm_blocks"],
        ttm_blocks=best.params["ttm_blocks"],
        rtm_heads= best.params["rtm_heads"],
        stm_heads= best.params["stm_heads"],
        ttm_heads= best.params["ttm_heads"],
        num_segments=best.params["num_segments"]
    ).to(DEVICE)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=best.params["lr"],
        weight_decay=best.params["weight_decay"]
    )
    criterion = nn.CrossEntropyLoss()

    # 3) MAX_EPOCHS 만큼 전체 학습하며 loss·accuracy 출력
    for epoch in range(1, MAX_EPOCHS + 1):
        model.train()
        loss_sum = 0.0
        correct  = 0
        total    = 0
        for X, y in full_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            logits = model(X)
            loss   = criterion(logits, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            correct  += (logits.argmax(1) == y).sum().item()
            total    += y.size(0)

        avg_loss = loss_sum / len(full_loader)
        acc      = correct / total
        print(f"[Full Train] Epoch {epoch:03d} | loss={avg_loss:.4f} | acc={acc:.4f}")

    # 4) 체크포인트 저장
    ckpt_dir = '/content/drive/MyDrive/2025_Lab_Research/checkpoints'
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, 'eegformer_best.pth')
    torch.save(model.state_dict(), ckpt_path)
    print(f"💾 Saved best model to {ckpt_path}")
